# 🧠 Retrieval-Augmented Generation (RAG) System
This notebook builds a **RAG pipeline** using FastAPI and ChromaDB.

It allows you to:
1. Upload up to 20 PDF or text files (each up to 1000 pages).
2. Split them into chunks and store in Chroma vector DB.
3. Ask questions and get context-based answers using GPT or Gemini.

---
**Run all cells below sequentially.**


In [ ]:
# Install dependencies
!pip install fastapi uvicorn python-multipart pydantic[dotenv] chromadb sentence-transformers PyPDF2 openai aiofiles nest_asyncio pyngrok --quiet

In [ ]:
# Imports and setup
import nest_asyncio, uvicorn, os
from pyngrok import ngrok
nest_asyncio.apply()

# Optional: set your OpenAI API key
# os.environ['OPENAI_API_KEY'] = 'sk-...'

# Start public tunnel
public_url = ngrok.connect(8000)
print('Public URL:', public_url)

# Save the app file
app_code = '''{  "nbformat": 4,  "nbformat_minor": 0,  "metadata": {    "colab": {      "provenance": [],      "gpuType": "T4"    },    "kernelspec": {      "name": "python3",      "display_name": "Python 3"    },    "language_info": {      "name": "python"    },    "accelerator": "GPU",    "widgets": {      "application/vnd.jupyter.widget-state+json": {        "f5108df1194646538efe27b2c502ec62": {          "model_module": "@jupyter-widgets/controls",          "model_name": "VBoxModel",          "model_module_version": "1.5.0",          "state": {            "_dom_classes": [],            "_model_module": "@jupyter-widgets/controls",            "_model_module_version": "1.5.0",            "_model_name": "VBoxModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/controls",            "_view_module_version": "1.5.0",            "_view_name": "VBoxView",            "box_style": "",            "children": [],            "layout": "IPY_MODEL_8418570205dc416083b062bf580345ae"          }        },        "0e4cdff0a2cd40e6bef99ae8153f1bfb": {          "model_module": "@jupyter-widgets/controls",          "model_name": "HTMLModel",          "model_module_version": "1.5.0",          "state": {            "_dom_classes": [],            "_model_module": "@jupyter-widgets/controls",            "_model_module_version": "1.5.0",            "_model_name": "HTMLModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/controls",            "_view_module_version": "1.5.0",            "_view_name": "HTMLView",            "description": "",            "description_tooltip": null,            "layout": "IPY_MODEL_f49da97483f844a49610208e9251866a",            "placeholder": "​",            "style": "IPY_MODEL_e22233b61029425d9c01f695672f5569",            "value": "<center> <img\nsrc=https://huggingface.co/front/assets/huggingface_logo-noborder.svg\nalt='Hugging Face'> <br> Copy a token from <a\nhref=\"https://huggingface.co/settings/tokens\" target=\"_blank\">your Hugging Face\ntokens page</a> and paste it below. <br> Immediately click login after copying\nyour token or it might be stored in plain text in this notebook file. </center>"          }        },        "45ede06fa69e4d8a8dde12c563c742e3": {          "model_module": "@jupyter-widgets/controls",          "model_name": "PasswordModel",          "model_module_version": "1.5.0",          "state": {            "_dom_classes": [],            "_model_module": "@jupyter-widgets/controls",            "_model_module_version": "1.5.0",            "_model_name": "PasswordModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/controls",            "_view_module_version": "1.5.0",            "_view_name": "PasswordView",            "continuous_update": true,            "description": "Token:",            "description_tooltip": null,            "disabled": false,            "layout": "IPY_MODEL_ba3b87ac7d7b4960a3fe138c05f99d46",            "placeholder": "​",            "style": "IPY_MODEL_2eda0349ffe84ccc8b89a0497bc124fe",            "value": ""          }        },        "8420565a9f0b459cb9a4763379b13851": {          "model_module": "@jupyter-widgets/controls",          "model_name": "CheckboxModel",          "model_module_version": "1.5.0",          "state": {            "_dom_classes": [],            "_model_module": "@jupyter-widgets/controls",            "_model_module_version": "1.5.0",            "_model_name": "CheckboxModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/controls",            "_view_module_version": "1.5.0",            "_view_name": "CheckboxView",            "description": "Add token as git credential?",            "description_tooltip": null,            "disabled": false,            "indent": true,            "layout": "IPY_MODEL_c67e328b579847379409cd470488184e",            "style": "IPY_MODEL_33b84329f06646859821d7494989758c",            "value": true          }        },        "83cf4129d1434849b8763fda880b0e2c": {          "model_module": "@jupyter-widgets/controls",          "model_name": "ButtonModel",          "model_module_version": "1.5.0",          "state": {            "_dom_classes": [],            "_model_module": "@jupyter-widgets/controls",            "_model_module_version": "1.5.0",            "_model_name": "ButtonModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/controls",            "_view_module_version": "1.5.0",            "_view_name": "ButtonView",            "button_style": "",            "description": "Login",            "disabled": false,            "icon": "",            "layout": "IPY_MODEL_3f272cbed3f949af9a6f961a83131ed1",            "style": "IPY_MODEL_36e19d253d6146a9aae40742044c5fa5",            "tooltip": ""          }        },        "e99236bcdb5d477f88be5bfd70cc32f9": {          "model_module": "@jupyter-widgets/controls",          "model_name": "HTMLModel",          "model_module_version": "1.5.0",          "state": {            "_dom_classes": [],            "_model_module": "@jupyter-widgets/controls",            "_model_module_version": "1.5.0",            "_model_name": "HTMLModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/controls",            "_view_module_version": "1.5.0",            "_view_name": "HTMLView",            "description": "",            "description_tooltip": null,            "layout": "IPY_MODEL_c210b08d990c4f4e94f23f3a152b8b83",            "placeholder": "​",            "style": "IPY_MODEL_ed6f78b90310486781bcaa82e9349098",            "value": "\n<b>Pro Tip:</b> If you don't already have one, you can create a dedicated\n'notebooks' token with 'write' access, that you can then easily reuse for all\nnotebooks. </center>"          }        },        "8418570205dc416083b062bf580345ae": {          "model_module": "@jupyter-widgets/base",          "model_name": "LayoutModel",          "model_module_version": "1.2.0",          "state": {            "_model_module": "@jupyter-widgets/base",            "_model_module_version": "1.2.0",            "_model_name": "LayoutModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/base",            "_view_module_version": "1.2.0",            "_view_name": "LayoutView",            "align_content": null,            "align_items": "center",            "align_self": null,            "border": null,            "bottom": null,            "display": "flex",            "flex": null,            "flex_flow": "column",            "grid_area": null,            "grid_auto_columns": null,            "grid_auto_flow": null,            "grid_auto_rows": null,            "grid_column": null,            "grid_gap": null,            "grid_row": null,            "grid_template_areas": null,            "grid_template_columns": null,            "grid_template_rows": null,            "height": null,            "justify_content": null,            "justify_items": null,            "left": null,            "margin": null,            "max_height": null,            "max_width": null,            "min_height": null,            "min_width": null,            "object_fit": null,            "object_position": null,            "order": null,            "overflow": null,            "overflow_x": null,            "overflow_y": null,            "padding": null,            "right": null,            "top": null,            "visibility": null,            "width": "50%"          }        },        "f49da97483f844a49610208e9251866a": {          "model_module": "@jupyter-widgets/base",          "model_name": "LayoutModel",          "model_module_version": "1.2.0",          "state": {            "_model_module": "@jupyter-widgets/base",            "_model_module_version": "1.2.0",            "_model_name": "LayoutModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/base",            "_view_module_version": "1.2.0",            "_view_name": "LayoutView",            "align_content": null,            "align_items": null,            "align_self": null,            "border": null,            "bottom": null,            "display": null,            "flex": null,            "flex_flow": null,            "grid_area": null,            "grid_auto_columns": null,            "grid_auto_flow": null,            "grid_auto_rows": null,            "grid_column": null,            "grid_gap": null,            "grid_row": null,            "grid_template_areas": null,            "grid_template_columns": null,            "grid_template_rows": null,            "height": null,            "justify_content": null,            "justify_items": null,            "left": null,            "margin": null,            "max_height": null,            "max_width": null,            "min_height": null,            "min_width": null,            "object_fit": null,            "object_position": null,            "order": null,            "overflow": null,            "overflow_x": null,            "overflow_y": null,            "padding": null,            "right": null,            "top": null,            "visibility": null,            "width": null          }        },        "e22233b61029425d9c01f695672f5569": {          "model_module": "@jupyter-widgets/controls",          "model_name": "DescriptionStyleModel",          "model_module_version": "1.5.0",          "state": {            "_model_module": "@jupyter-widgets/controls",            "_model_module_version": "1.5.0",            "_model_name": "DescriptionStyleModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/base",            "_view_module_version": "1.2.0",            "_view_name": "StyleView",            "description_width": ""          }        },        "ba3b87ac7d7b4960a3fe138c05f99d46": {          "model_module": "@jupyter-widgets/base",          "model_name": "LayoutModel",          "model_module_version": "1.2.0",          "state": {            "_model_module": "@jupyter-widgets/base",            "_model_module_version": "1.2.0",            "_model_name": "LayoutModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/base",            "_view_module_version": "1.2.0",            "_view_name": "LayoutView",            "align_content": null,            "align_items": null,            "align_self": null,            "border": null,            "bottom": null,            "display": null,            "flex": null,            "flex_flow": null,            "grid_area": null,            "grid_auto_columns": null,            "grid_auto_flow": null,            "grid_auto_rows": null,            "grid_column": null,            "grid_gap": null,            "grid_row": null,            "grid_template_areas": null,            "grid_template_columns": null,            "grid_template_rows": null,            "height": null,            "justify_content": null,            "justify_items": null,            "left": null,            "margin": null,            "max_height": null,            "max_width": null,            "min_height": null,            "min_width": null,            "object_fit": null,            "object_position": null,            "order": null,            "overflow": null,            "overflow_x": null,            "overflow_y": null,            "padding": null,            "right": null,            "top": null,            "visibility": null,            "width": null          }        },        "2eda0349ffe84ccc8b89a0497bc124fe": {          "model_module": "@jupyter-widgets/controls",          "model_name": "DescriptionStyleModel",          "model_module_version": "1.5.0",          "state": {            "_model_module": "@jupyter-widgets/controls",            "_model_module_version": "1.5.0",            "_model_name": "DescriptionStyleModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/base",            "_view_module_version": "1.2.0",            "_view_name": "StyleView",            "description_width": ""          }        },        "c67e328b579847379409cd470488184e": {          "model_module": "@jupyter-widgets/base",          "model_name": "LayoutModel",          "model_module_version": "1.2.0",          "state": {            "_model_module": "@jupyter-widgets/base",            "_model_module_version": "1.2.0",            "_model_name": "LayoutModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/base",            "_view_module_version": "1.2.0",            "_view_name": "LayoutView",            "align_content": null,            "align_items": null,            "align_self": null,            "border": null,            "bottom": null,            "display": null,            "flex": null,            "flex_flow": null,            "grid_area": null,            "grid_auto_columns": null,            "grid_auto_flow": null,            "grid_auto_rows": null,            "grid_column": null,            "grid_gap": null,            "grid_row": null,            "grid_template_areas": null,            "grid_template_columns": null,            "grid_template_rows": null,            "height": null,            "justify_content": null,            "justify_items": null,            "left": null,            "margin": null,            "max_height": null,            "max_width": null,            "min_height": null,            "min_width": null,            "object_fit": null,            "object_position": null,            "order": null,            "overflow": null,            "overflow_x": null,            "overflow_y": null,            "padding": null,            "right": null,            "top": null,            "visibility": null,            "width": null          }        },        "33b84329f06646859821d7494989758c": {          "model_module": "@jupyter-widgets/controls",          "model_name": "DescriptionStyleModel",          "model_module_version": "1.5.0",          "state": {            "_model_module": "@jupyter-widgets/controls",            "_model_module_version": "1.5.0",            "_model_name": "DescriptionStyleModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/base",            "_view_module_version": "1.2.0",            "_view_name": "StyleView",            "description_width": ""          }        },        "3f272cbed3f949af9a6f961a83131ed1": {          "model_module": "@jupyter-widgets/base",          "model_name": "LayoutModel",          "model_module_version": "1.2.0",          "state": {            "_model_module": "@jupyter-widgets/base",            "_model_module_version": "1.2.0",            "_model_name": "LayoutModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/base",            "_view_module_version": "1.2.0",            "_view_name": "LayoutView",            "align_content": null,            "align_items": null,            "align_self": null,            "border": null,            "bottom": null,            "display": null,            "flex": null,            "flex_flow": null,            "grid_area": null,            "grid_auto_columns": null,            "grid_auto_flow": null,            "grid_auto_rows": null,            "grid_column": null,            "grid_gap": null,            "grid_row": null,            "grid_template_areas": null,            "grid_template_columns": null,            "grid_template_rows": null,            "height": null,            "justify_content": null,            "justify_items": null,            "left": null,            "margin": null,            "max_height": null,            "max_width": null,            "min_height": null,            "min_width": null,            "object_fit": null,            "object_position": null,            "order": null,            "overflow": null,            "overflow_x": null,            "overflow_y": null,            "padding": null,            "right": null,            "top": null,            "visibility": null,            "width": null          }        },        "36e19d253d6146a9aae40742044c5fa5": {          "model_module": "@jupyter-widgets/controls",          "model_name": "ButtonStyleModel",          "model_module_version": "1.5.0",          "state": {            "_model_module": "@jupyter-widgets/controls",            "_model_module_version": "1.5.0",            "_model_name": "ButtonStyleModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/base",            "_view_module_version": "1.2.0",            "_view_name": "StyleView",            "button_color": null,            "font_weight": ""          }        },        "c210b08d990c4f4e94f23f3a152b8b83": {          "model_module": "@jupyter-widgets/base",          "model_name": "LayoutModel",          "model_module_version": "1.2.0",          "state": {            "_model_module": "@jupyter-widgets/base",            "_model_module_version": "1.2.0",            "_model_name": "LayoutModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/base",            "_view_module_version": "1.2.0",            "_view_name": "LayoutView",            "align_content": null,            "align_items": null,            "align_self": null,            "border": null,            "bottom": null,            "display": null,            "flex": null,            "flex_flow": null,            "grid_area": null,            "grid_auto_columns": null,            "grid_auto_flow": null,            "grid_auto_rows": null,            "grid_column": null,            "grid_gap": null,            "grid_row": null,            "grid_template_areas": null,            "grid_template_columns": null,            "grid_template_rows": null,            "height": null,            "justify_content": null,            "justify_items": null,            "left": null,            "margin": null,            "max_height": null,            "max_width": null,            "min_height": null,            "min_width": null,            "object_fit": null,            "object_position": null,            "order": null,            "overflow": null,            "overflow_x": null,            "overflow_y": null,            "padding": null,            "right": null,            "top": null,            "visibility": null,            "width": null          }        },        "ed6f78b90310486781bcaa82e9349098": {          "model_module": "@jupyter-widgets/controls",          "model_name": "DescriptionStyleModel",          "model_module_version": "1.5.0",          "state": {            "_model_module": "@jupyter-widgets/controls",            "_model_module_version": "1.5.0",            "_model_name": "DescriptionStyleModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/base",            "_view_module_version": "1.2.0",            "_view_name": "StyleView",            "description_width": ""          }        },        "8d1cda37e82b4d20bf885aadcb6b15d4": {          "model_module": "@jupyter-widgets/controls",          "model_name": "LabelModel",          "model_module_version": "1.5.0",          "state": {            "_dom_classes": [],            "_model_module": "@jupyter-widgets/controls",            "_model_module_version": "1.5.0",            "_model_name": "LabelModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/controls",            "_view_module_version": "1.5.0",            "_view_name": "LabelView",            "description": "",            "description_tooltip": null,            "layout": "IPY_MODEL_9d33802ae3274dc8aa9d796a379c03c9",            "placeholder": "​",            "style": "IPY_MODEL_eafdfe329aa747e7833841e11b141183",            "value": "Connecting..."          }        },        "9d33802ae3274dc8aa9d796a379c03c9": {          "model_module": "@jupyter-widgets/base",          "model_name": "LayoutModel",          "model_module_version": "1.2.0",          "state": {            "_model_module": "@jupyter-widgets/base",            "_model_module_version": "1.2.0",            "_model_name": "LayoutModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/base",            "_view_module_version": "1.2.0",            "_view_name": "LayoutView",            "align_content": null,            "align_items": null,            "align_self": null,            "border": null,            "bottom": null,            "display": null,            "flex": null,            "flex_flow": null,            "grid_area": null,            "grid_auto_columns": null,            "grid_auto_flow": null,            "grid_auto_rows": null,            "grid_column": null,            "grid_gap": null,            "grid_row": null,            "grid_template_areas": null,            "grid_template_columns": null,            "grid_template_rows": null,            "height": null,            "justify_content": null,            "justify_items": null,            "left": null,            "margin": null,            "max_height": null,            "max_width": null,            "min_height": null,            "min_width": null,            "object_fit": null,            "object_position": null,            "order": null,            "overflow": null,            "overflow_x": null,            "overflow_y": null,            "padding": null,            "right": null,            "top": null,            "visibility": null,            "width": null          }        },        "eafdfe329aa747e7833841e11b141183": {          "model_module": "@jupyter-widgets/controls",          "model_name": "DescriptionStyleModel",          "model_module_version": "1.5.0",          "state": {            "_model_module": "@jupyter-widgets/controls",            "_model_module_version": "1.5.0",            "_model_name": "DescriptionStyleModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/base",            "_view_module_version": "1.2.0",            "_view_name": "StyleView",            "description_width": ""          }        },        "6187d9bc3315463599b6c057cdbb6677": {          "model_module": "@jupyter-widgets/controls",          "model_name": "HBoxModel",          "model_module_version": "1.5.0",          "state": {            "_dom_classes": [],            "_model_module": "@jupyter-widgets/controls",            "_model_module_version": "1.5.0",            "_model_name": "HBoxModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/controls",            "_view_module_version": "1.5.0",            "_view_name": "HBoxView",            "box_style": "",            "children": [              "IPY_MODEL_d7c6b661e272463cbc56b53f87f33abc",              "IPY_MODEL_882cd1317ae646ab85f7ed7807d84414",              "IPY_MODEL_7eef6b642dc84be5bed85901c52b8b9a"            ],            "layout": "IPY_MODEL_bc72f7048f9544b6b2531714f1ff83f0"          }        },        "d7c6b661e272463cbc56b53f87f33abc": {          "model_module": "@jupyter-widgets/controls",          "model_name": "HTMLModel",          "model_module_version": "1.5.0",          "state": {            "_dom_classes": [],            "_model_module": "@jupyter-widgets/controls",            "_model_module_version": "1.5.0",            "_model_name": "HTMLModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/controls",            "_view_module_version": "1.5.0",            "_view_name": "HTMLView",            "description": "",            "description_tooltip": null,            "layout": "IPY_MODEL_a9a1dbbe6c164a0cb7fca4285085fefc",            "placeholder": "​",            "style": "IPY_MODEL_720f0bf7849a46c8b62ec266894aee9c",            "value": "modules.json: 100%"          }        },        "882cd1317ae646ab85f7ed7807d84414": {          "model_module": "@jupyter-widgets/controls",          "model_name": "FloatProgressModel",          "model_module_version": "1.5.0",          "state": {            "_dom_classes": [],            "_model_module": "@jupyter-widgets/controls",            "_model_module_version": "1.5.0",            "_model_name": "FloatProgressModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/controls",            "_view_module_version": "1.5.0",            "_view_name": "ProgressView",            "bar_style": "success",            "description": "",            "description_tooltip": null,            "layout": "IPY_MODEL_db038b06c48f4c5cb66aec0f7d151e5d",            "max": 461,            "min": 0,            "orientation": "horizontal",            "style": "IPY_MODEL_eaa06dbf3355419eb3fbd8830d380b6c",            "value": 461          }        },        "7eef6b642dc84be5bed85901c52b8b9a": {          "model_module": "@jupyter-widgets/controls",          "model_name": "HTMLModel",          "model_module_version": "1.5.0",          "state": {            "_dom_classes": [],            "_model_module": "@jupyter-widgets/controls",            "_model_module_version": "1.5.0",            "_model_name": "HTMLModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/controls",            "_view_module_version": "1.5.0",            "_view_name": "HTMLView",            "description": "",            "description_tooltip": null,            "layout": "IPY_MODEL_973b635b6bb64b16b51efec17e43d206",            "placeholder": "​",            "style": "IPY_MODEL_6a1e95ad9bae4feaaddf65b2456e4423",            "value": " 461/461 [00:00&lt;00:00, 34.5kB/s]"          }        },        "bc72f7048f9544b6b2531714f1ff83f0": {          "model_module": "@jupyter-widgets/base",          "model_name": "LayoutModel",          "model_module_version": "1.2.0",          "state": {            "_model_module": "@jupyter-widgets/base",            "_model_module_version": "1.2.0",            "_model_name": "LayoutModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/base",            "_view_module_version": "1.2.0",            "_view_name": "LayoutView",            "align_content": null,            "align_items": null,            "align_self": null,            "border": null,            "bottom": null,            "display": null,            "flex": null,            "flex_flow": null,            "grid_area": null,            "grid_auto_columns": null,            "grid_auto_flow": null,            "grid_auto_rows": null,            "grid_column": null,            "grid_gap": null,            "grid_row": null,            "grid_template_areas": null,            "grid_template_columns": null,            "grid_template_rows": null,            "height": null,            "justify_content": null,            "justify_items": null,            "left": null,            "margin": null,            "max_height": null,            "max_width": null,            "min_height": null,            "min_width": null,            "object_fit": null,            "object_position": null,            "order": null,            "overflow": null,            "overflow_x": null,            "overflow_y": null,            "padding": null,            "right": null,            "top": null,            "visibility": null,            "width": null          }        },        "a9a1dbbe6c164a0cb7fca4285085fefc": {          "model_module": "@jupyter-widgets/base",          "model_name": "LayoutModel",          "model_module_version": "1.2.0",          "state": {            "_model_module": "@jupyter-widgets/base",            "_model_module_version": "1.2.0",            "_model_name": "LayoutModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/base",            "_view_module_version": "1.2.0",            "_view_name": "LayoutView",            "align_content": null,            "align_items": null,            "align_self": null,            "border": null,            "bottom": null,            "display": null,            "flex": null,            "flex_flow": null,            "grid_area": null,            "grid_auto_columns": null,            "grid_auto_flow": null,            "grid_auto_rows": null,            "grid_column": null,            "grid_gap": null,            "grid_row": null,            "grid_template_areas": null,            "grid_template_columns": null,            "grid_template_rows": null,            "height": null,            "justify_content": null,            "justify_items": null,            "left": null,            "margin": null,            "max_height": null,            "max_width": null,            "min_height": null,            "min_width": null,            "object_fit": null,            "object_position": null,            "order": null,            "overflow": null,            "overflow_x": null,            "overflow_y": null,            "padding": null,            "right": null,            "top": null,            "visibility": null,            "width": null          }        },        "720f0bf7849a46c8b62ec266894aee9c": {          "model_module": "@jupyter-widgets/controls",          "model_name": "DescriptionStyleModel",          "model_module_version": "1.5.0",          "state": {            "_model_module": "@jupyter-widgets/controls",            "_model_module_version": "1.5.0",            "_model_name": "DescriptionStyleModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/base",            "_view_module_version": "1.2.0",            "_view_name": "StyleView",            "description_width": ""          }        },        "db038b06c48f4c5cb66aec0f7d151e5d": {          "model_module": "@jupyter-widgets/base",          "model_name": "LayoutModel",          "model_module_version": "1.2.0",          "state": {            "_model_module": "@jupyter-widgets/base",            "_model_module_version": "1.2.0",            "_model_name": "LayoutModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/base",            "_view_module_version": "1.2.0",            "_view_name": "LayoutView",            "align_content": null,            "align_items": null,            "align_self": null,            "border": null,            "bottom": null,            "display": null,            "flex": null,            "flex_flow": null,            "grid_area": null,            "grid_auto_columns": null,            "grid_auto_flow": null,            "grid_auto_rows": null,            "grid_column": null,            "grid_gap": null,            "grid_row": null,            "grid_template_areas": null,            "grid_template_columns": null,            "grid_template_rows": null,            "height": null,            "justify_content": null,            "justify_items": null,            "left": null,            "margin": null,            "max_height": null,            "max_width": null,            "min_height": null,            "min_width": null,            "object_fit": null,            "object_position": null,            "order": null,            "overflow": null,            "overflow_x": null,            "overflow_y": null,            "padding": null,            "right": null,            "top": null,            "visibility": null,            "width": null          }        },        "eaa06dbf3355419eb3fbd8830d380b6c": {          "model_module": "@jupyter-widgets/controls",          "model_name": "ProgressStyleModel",          "model_module_version": "1.5.0",          "state": {            "_model_module": "@jupyter-widgets/controls",            "_model_module_version": "1.5.0",            "_model_name": "ProgressStyleModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/base",            "_view_module_version": "1.2.0",            "_view_name": "StyleView",            "bar_color": null,            "description_width": ""          }        },        "973b635b6bb64b16b51efec17e43d206": {          "model_module": "@jupyter-widgets/base",          "model_name": "LayoutModel",          "model_module_version": "1.2.0",          "state": {            "_model_module": "@jupyter-widgets/base",            "_model_module_version": "1.2.0",            "_model_name": "LayoutModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/base",            "_view_module_version": "1.2.0",            "_view_name": "LayoutView",            "align_content": null,            "align_items": null,            "align_self": null,            "border": null,            "bottom": null,            "display": null,            "flex": null,            "flex_flow": null,            "grid_area": null,            "grid_auto_columns": null,            "grid_auto_flow": null,            "grid_auto_rows": null,            "grid_column": null,            "grid_gap": null,            "grid_row": null,            "grid_template_areas": null,            "grid_template_columns": null,            "grid_template_rows": null,            "height": null,            "justify_content": null,            "justify_items": null,            "left": null,            "margin": null,            "max_height": null,            "max_width": null,            "min_height": null,            "min_width": null,            "object_fit": null,            "object_position": null,            "order": null,            "overflow": null,            "overflow_x": null,            "overflow_y": null,            "padding": null,            "right": null,            "top": null,            "visibility": null,            "width": null          }        },        "6a1e95ad9bae4feaaddf65b2456e4423": {          "model_module": "@jupyter-widgets/controls",          "model_name": "DescriptionStyleModel",          "model_module_version": "1.5.0",          "state": {            "_model_module": "@jupyter-widgets/controls",            "_model_module_version": "1.5.0",            "_model_name": "DescriptionStyleModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/base",            "_view_module_version": "1.2.0",            "_view_name": "StyleView",            "description_width": ""          }        },        "30e40062550f4fa0ae0f384d2925ece6": {          "model_module": "@jupyter-widgets/controls",          "model_name": "HBoxModel",          "model_module_version": "1.5.0",          "state": {            "_dom_classes": [],            "_model_module": "@jupyter-widgets/controls",            "_model_module_version": "1.5.0",            "_model_name": "HBoxModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/controls",            "_view_module_version": "1.5.0",            "_view_name": "HBoxView",            "box_style": "",            "children": [              "IPY_MODEL_7610f4069f4e4daeaff9b986e67b0d1f",              "IPY_MODEL_18314b1870f24774afb973fdbd03b491",              "IPY_MODEL_bff75da6cd2a41c9b42bd0e398070311"            ],            "layout": "IPY_MODEL_ba0259a97a2248eaa48d42453156bfc8"          }        },        "7610f4069f4e4daeaff9b986e67b0d1f": {          "model_module": "@jupyter-widgets/controls",          "model_name": "HTMLModel",          "model_module_version": "1.5.0",          "state": {            "_dom_classes": [],            "_model_module": "@jupyter-widgets/controls",            "_model_module_version": "1.5.0",            "_model_name": "HTMLModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/controls",            "_view_module_version": "1.5.0",            "_view_name": "HTMLView",            "description": "",            "description_tooltip": null,            "layout": "IPY_MODEL_4b4ec49da1224c79bd9b48dadf32aba1",            "placeholder": "​",            "style": "IPY_MODEL_79087128df004e36a6fa6555c701fed7",            "value": "config_sentence_transformers.json: 100%"          }        },        "18314b1870f24774afb973fdbd03b491": {          "model_module": "@jupyter-widgets/controls",          "model_name": "FloatProgressModel",          "model_module_version": "1.5.0",          "state": {            "_dom_classes": [],            "_model_module": "@jupyter-widgets/controls",            "_model_module_version": "1.5.0",            "_model_name": "FloatProgressModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/controls",            "_view_module_version": "1.5.0",            "_view_name": "ProgressView",            "bar_style": "success",            "description": "",            "description_tooltip": null,            "layout": "IPY_MODEL_2cf5615646ab44be81bb5d24dcd71ffa",            "max": 122,            "min": 0,            "orientation": "horizontal",            "style": "IPY_MODEL_9e5363a9269a4db19e89217c69f71bc9",            "value": 122          }        },        "bff75da6cd2a41c9b42bd0e398070311": {          "model_module": "@jupyter-widgets/controls",          "model_name": "HTMLModel",          "model_module_version": "1.5.0",          "state": {            "_dom_classes": [],            "_model_module": "@jupyter-widgets/controls",            "_model_module_version": "1.5.0",            "_model_name": "HTMLModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/controls",            "_view_module_version": "1.5.0",            "_view_name": "HTMLView",            "description": "",            "description_tooltip": null,            "layout": "IPY_MODEL_5546306cb5d44e4aa1990e16cc528cfb",            "placeholder": "​",            "style": "IPY_MODEL_b581fb6120f44472b280ee21245b8a2f",            "value": " 122/122 [00:00&lt;00:00, 9.28kB/s]"          }        },        "ba0259a97a2248eaa48d42453156bfc8": {          "model_module": "@jupyter-widgets/base",          "model_name": "LayoutModel",          "model_module_version": "1.2.0",          "state": {            "_model_module": "@jupyter-widgets/base",            "_model_module_version": "1.2.0",            "_model_name": "LayoutModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/base",            "_view_module_version": "1.2.0",            "_view_name": "LayoutView",            "align_content": null,            "align_items": null,            "align_self": null,            "border": null,            "bottom": null,            "display": null,            "flex": null,            "flex_flow": null,            "grid_area": null,            "grid_auto_columns": null,            "grid_auto_flow": null,            "grid_auto_rows": null,            "grid_column": null,            "grid_gap": null,            "grid_row": null,            "grid_template_areas": null,            "grid_template_columns": null,            "grid_template_rows": null,            "height": null,            "justify_content": null,            "justify_items": null,            "left": null,            "margin": null,            "max_height": null,            "max_width": null,            "min_height": null,            "min_width": null,            "object_fit": null,            "object_position": null,            "order": null,            "overflow": null,            "overflow_x": null,            "overflow_y": null,            "padding": null,            "right": null,            "top": null,            "visibility": null,            "width": null          }        },        "4b4ec49da1224c79bd9b48dadf32aba1": {          "model_module": "@jupyter-widgets/base",          "model_name": "LayoutModel",          "model_module_version": "1.2.0",          "state": {            "_model_module": "@jupyter-widgets/base",            "_model_module_version": "1.2.0",            "_model_name": "LayoutModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/base",            "_view_module_version": "1.2.0",            "_view_name": "LayoutView",            "align_content": null,            "align_items": null,            "align_self": null,            "border": null,            "bottom": null,            "display": null,            "flex": null,            "flex_flow": null,            "grid_area": null,            "grid_auto_columns": null,            "grid_auto_flow": null,            "grid_auto_rows": null,            "grid_column": null,            "grid_gap": null,            "grid_row": null,            "grid_template_areas": null,            "grid_template_columns": null,            "grid_template_rows": null,            "height": null,            "justify_content": null,            "justify_items": null,            "left": null,            "margin": null,            "max_height": null,            "max_width": null,            "min_height": null,            "min_width": null,            "object_fit": null,            "object_position": null,            "order": null,            "overflow": null,            "overflow_x": null,            "overflow_y": null,            "padding": null,            "right": null,            "top": null,            "visibility": null,            "width": null          }        },        "79087128df004e36a6fa6555c701fed7": {          "model_module": "@jupyter-widgets/controls",          "model_name": "DescriptionStyleModel",          "model_module_version": "1.5.0",          "state": {            "_model_module": "@jupyter-widgets/controls",            "_model_module_version": "1.5.0",            "_model_name": "DescriptionStyleModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/base",            "_view_module_version": "1.2.0",            "_view_name": "StyleView",            "description_width": ""          }        },        "2cf5615646ab44be81bb5d24dcd71ffa": {          "model_module": "@jupyter-widgets/base",          "model_name": "LayoutModel",          "model_module_version": "1.2.0",          "state": {            "_model_module": "@jupyter-widgets/base",            "_model_module_version": "1.2.0",            "_model_name": "LayoutModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/base",            "_view_module_version": "1.2.0",            "_view_name": "LayoutView",            "align_content": null,            "align_items": null,            "align_self": null,            "border": null,            "bottom": null,            "display": null,            "flex": null,            "flex_flow": null,            "grid_area": null,            "grid_auto_columns": null,            "grid_auto_flow": null,            "grid_auto_rows": null,            "grid_column": null,            "grid_gap": null,            "grid_row": null,            "grid_template_areas": null,            "grid_template_columns": null,            "grid_template_rows": null,            "height": null,            "justify_content": null,            "justify_items": null,            "left": null,            "margin": null,            "max_height": null,            "max_width": null,            "min_height": null,            "min_width": null,            "object_fit": null,            "object_position": null,            "order": null,            "overflow": null,            "overflow_x": null,            "overflow_y": null,            "padding": null,            "right": null,            "top": null,            "visibility": null,            "width": null          }        },        "9e5363a9269a4db19e89217c69f71bc9": {          "model_module": "@jupyter-widgets/controls",          "model_name": "ProgressStyleModel",          "model_module_version": "1.5.0",          "state": {            "_model_module": "@jupyter-widgets/controls",            "_model_module_version": "1.5.0",            "_model_name": "ProgressStyleModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/base",            "_view_module_version": "1.2.0",            "_view_name": "StyleView",            "bar_color": null,            "description_width": ""          }        },        "5546306cb5d44e4aa1990e16cc528cfb": {          "model_module": "@jupyter-widgets/base",          "model_name": "LayoutModel",          "model_module_version": "1.2.0",          "state": {            "_model_module": "@jupyter-widgets/base",            "_model_module_version": "1.2.0",            "_model_name": "LayoutModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/base",            "_view_module_version": "1.2.0",            "_view_name": "LayoutView",            "align_content": null,            "align_items": null,            "align_self": null,            "border": null,            "bottom": null,            "display": null,            "flex": null,            "flex_flow": null,            "grid_area": null,            "grid_auto_columns": null,            "grid_auto_flow": null,            "grid_auto_rows": null,            "grid_column": null,            "grid_gap": null,            "grid_row": null,            "grid_template_areas": null,            "grid_template_columns": null,            "grid_template_rows": null,            "height": null,            "justify_content": null,            "justify_items": null,            "left": null,            "margin": null,            "max_height": null,            "max_width": null,            "min_height": null,            "min_width": null,            "object_fit": null,            "object_position": null,            "order": null,            "overflow": null,            "overflow_x": null,            "overflow_y": null,            "padding": null,            "right": null,            "top": null,            "visibility": null,            "width": null          }        },        "b581fb6120f44472b280ee21245b8a2f": {          "model_module": "@jupyter-widgets/controls",          "model_name": "DescriptionStyleModel",          "model_module_version": "1.5.0",          "state": {            "_model_module": "@jupyter-widgets/controls",            "_model_module_version": "1.5.0",            "_model_name": "DescriptionStyleModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/base",            "_view_module_version": "1.2.0",            "_view_name": "StyleView",            "description_width": ""          }        },        "ed5f0f474a724d9787bb869c8858660b": {          "model_module": "@jupyter-widgets/controls",          "model_name": "HBoxModel",          "model_module_version": "1.5.0",          "state": {            "_dom_classes": [],            "_model_module": "@jupyter-widgets/controls",            "_model_module_version": "1.5.0",            "_model_name": "HBoxModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/controls",            "_view_module_version": "1.5.0",            "_view_name": "HBoxView",            "box_style": "",            "children": [              "IPY_MODEL_a81606e7465b4b539f1adb711b60a0b5",              "IPY_MODEL_d16e34e14f72466fb71aec1afb7c6ae9",              "IPY_MODEL_1e76695042ff447a9ee524a332edf1d6"            ],            "layout": "IPY_MODEL_a658ff4f08934270838e0a0d8d81ebf4"          }        },        "a81606e7465b4b539f1adb711b60a0b5": {          "model_module": "@jupyter-widgets/controls",          "model_name": "HTMLModel",          "model_module_version": "1.5.0",          "state": {            "_dom_classes": [],            "_model_module": "@jupyter-widgets/controls",            "_model_module_version": "1.5.0",            "_model_name": "HTMLModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/controls",            "_view_module_version": "1.5.0",            "_view_name": "HTMLView",            "description": "",            "description_tooltip": null,            "layout": "IPY_MODEL_687bd28291504e7db61ec00f381b056a",            "placeholder": "​",            "style": "IPY_MODEL_39c24019a24b47ca9cc22a549ed2c92d",            "value": "README.md: "          }        },        "d16e34e14f72466fb71aec1afb7c6ae9": {          "model_module": "@jupyter-widgets/controls",          "model_name": "FloatProgressModel",          "model_module_version": "1.5.0",          "state": {            "_dom_classes": [],            "_model_module": "@jupyter-widgets/controls",            "_model_module_version": "1.5.0",            "_model_name": "FloatProgressModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/controls",            "_view_module_version": "1.5.0",            "_view_name": "ProgressView",            "bar_style": "success",            "description": "",            "description_tooltip": null,            "layout": "IPY_MODEL_420af6cbf08e4143945b60c5177b38b4",            "max": 1,            "min": 0,            "orientation": "horizontal",            "style": "IPY_MODEL_ccc0e88464bf43e8bde63966e7ae9c57",            "value": 1          }        },        "1e76695042ff447a9ee524a332edf1d6": {          "model_module": "@jupyter-widgets/controls",          "model_name": "HTMLModel",          "model_module_version": "1.5.0",          "state": {            "_dom_classes": [],            "_model_module": "@jupyter-widgets/controls",            "_model_module_version": "1.5.0",            "_model_name": "HTMLModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/controls",            "_view_module_version": "1.5.0",            "_view_name": "HTMLView",            "description": "",            "description_tooltip": null,            "layout": "IPY_MODEL_7f3739e5807c4a1484801a6d231b4b1c",            "placeholder": "​",            "style": "IPY_MODEL_d4fb0b35c6954c7dab1ed5becb8c8141",            "value": " 66.3k/? [00:00&lt;00:00, 5.08MB/s]"          }        },        "a658ff4f08934270838e0a0d8d81ebf4": {          "model_module": "@jupyter-widgets/base",          "model_name": "LayoutModel",          "model_module_version": "1.2.0",          "state": {            "_model_module": "@jupyter-widgets/base",            "_model_module_version": "1.2.0",            "_model_name": "LayoutModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/base",            "_view_module_version": "1.2.0",            "_view_name": "LayoutView",            "align_content": null,            "align_items": null,            "align_self": null,            "border": null,            "bottom": null,            "display": null,            "flex": null,            "flex_flow": null,            "grid_area": null,            "grid_auto_columns": null,            "grid_auto_flow": null,            "grid_auto_rows": null,            "grid_column": null,            "grid_gap": null,            "grid_row": null,            "grid_template_areas": null,            "grid_template_columns": null,            "grid_template_rows": null,            "height": null,            "justify_content": null,            "justify_items": null,            "left": null,            "margin": null,            "max_height": null,            "max_width": null,            "min_height": null,            "min_width": null,            "object_fit": null,            "object_position": null,            "order": null,            "overflow": null,            "overflow_x": null,            "overflow_y": null,            "padding": null,            "right": null,            "top": null,            "visibility": null,            "width": null          }        },        "687bd28291504e7db61ec00f381b056a": {          "model_module": "@jupyter-widgets/base",          "model_name": "LayoutModel",          "model_module_version": "1.2.0",          "state": {            "_model_module": "@jupyter-widgets/base",            "_model_module_version": "1.2.0",            "_model_name": "LayoutModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/base",            "_view_module_version": "1.2.0",            "_view_name": "LayoutView",            "align_content": null,            "align_items": null,            "align_self": null,            "border": null,            "bottom": null,            "display": null,            "flex": null,            "flex_flow": null,            "grid_area": null,            "grid_auto_columns": null,            "grid_auto_flow": null,            "grid_auto_rows": null,            "grid_column": null,            "grid_gap": null,            "grid_row": null,            "grid_template_areas": null,            "grid_template_columns": null,            "grid_template_rows": null,            "height": null,            "justify_content": null,            "justify_items": null,            "left": null,            "margin": null,            "max_height": null,            "max_width": null,            "min_height": null,            "min_width": null,            "object_fit": null,            "object_position": null,            "order": null,            "overflow": null,            "overflow_x": null,            "overflow_y": null,            "padding": null,            "right": null,            "top": null,            "visibility": null,            "width": null          }        },        "39c24019a24b47ca9cc22a549ed2c92d": {          "model_module": "@jupyter-widgets/controls",          "model_name": "DescriptionStyleModel",          "model_module_version": "1.5.0",          "state": {            "_model_module": "@jupyter-widgets/controls",            "_model_module_version": "1.5.0",            "_model_name": "DescriptionStyleModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/base",            "_view_module_version": "1.2.0",            "_view_name": "StyleView",            "description_width": ""          }        },        "420af6cbf08e4143945b60c5177b38b4": {          "model_module": "@jupyter-widgets/base",          "model_name": "LayoutModel",          "model_module_version": "1.2.0",          "state": {            "_model_module": "@jupyter-widgets/base",            "_model_module_version": "1.2.0",            "_model_name": "LayoutModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/base",            "_view_module_version": "1.2.0",            "_view_name": "LayoutView",            "align_content": null,            "align_items": null,            "align_self": null,            "border": null,            "bottom": null,            "display": null,            "flex": null,            "flex_flow": null,            "grid_area": null,            "grid_auto_columns": null,            "grid_auto_flow": null,            "grid_auto_rows": null,            "grid_column": null,            "grid_gap": null,            "grid_row": null,            "grid_template_areas": null,            "grid_template_columns": null,            "grid_template_rows": null,            "height": null,            "justify_content": null,            "justify_items": null,            "left": null,            "margin": null,            "max_height": null,            "max_width": null,            "min_height": null,            "min_width": null,            "object_fit": null,            "object_position": null,            "order": null,            "overflow": null,            "overflow_x": null,            "overflow_y": null,            "padding": null,            "right": null,            "top": null,            "visibility": null,            "width": "20px"          }        },        "ccc0e88464bf43e8bde63966e7ae9c57": {          "model_module": "@jupyter-widgets/controls",          "model_name": "ProgressStyleModel",          "model_module_version": "1.5.0",          "state": {            "_model_module": "@jupyter-widgets/controls",            "_model_module_version": "1.5.0",            "_model_name": "ProgressStyleModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/base",            "_view_module_version": "1.2.0",            "_view_name": "StyleView",            "bar_color": null,            "description_width": ""          }        },        "7f3739e5807c4a1484801a6d231b4b1c": {          "model_module": "@jupyter-widgets/base",          "model_name": "LayoutModel",          "model_module_version": "1.2.0",          "state": {            "_model_module": "@jupyter-widgets/base",            "_model_module_version": "1.2.0",            "_model_name": "LayoutModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/base",            "_view_module_version": "1.2.0",            "_view_name": "LayoutView",            "align_content": null,            "align_items": null,            "align_self": null,            "border": null,            "bottom": null,            "display": null,            "flex": null,            "flex_flow": null,            "grid_area": null,            "grid_auto_columns": null,            "grid_auto_flow": null,            "grid_auto_rows": null,            "grid_column": null,            "grid_gap": null,            "grid_row": null,            "grid_template_areas": null,            "grid_template_columns": null,            "grid_template_rows": null,            "height": null,            "justify_content": null,            "justify_items": null,            "left": null,            "margin": null,            "max_height": null,            "max_width": null,            "min_height": null,            "min_width": null,            "object_fit": null,            "object_position": null,            "order": null,            "overflow": null,            "overflow_x": null,            "overflow_y": null,            "padding": null,            "right": null,            "top": null,            "visibility": null,            "width": null          }        },        "d4fb0b35c6954c7dab1ed5becb8c8141": {          "model_module": "@jupyter-widgets/controls",          "model_name": "DescriptionStyleModel",          "model_module_version": "1.5.0",          "state": {            "_model_module": "@jupyter-widgets/controls",            "_model_module_version": "1.5.0",            "_model_name": "DescriptionStyleModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/base",            "_view_module_version": "1.2.0",            "_view_name": "StyleView",            "description_width": ""          }        },        "acd1bcda3bca4c30a9787f47f6267383": {          "model_module": "@jupyter-widgets/controls",          "model_name": "HBoxModel",          "model_module_version": "1.5.0",          "state": {            "_dom_classes": [],            "_model_module": "@jupyter-widgets/controls",            "_model_module_version": "1.5.0",            "_model_name": "HBoxModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/controls",            "_view_module_version": "1.5.0",            "_view_name": "HBoxView",            "box_style": "",            "children": [              "IPY_MODEL_d3830d8299cb427f98d210867c40b03e",              "IPY_MODEL_b17e87e86068463b81823634d795ad4d",              "IPY_MODEL_c23381aba2b54442840843488d2b3b34"            ],            "layout": "IPY_MODEL_b8e1668a7dbc41939b6cdd1b7ed55050"          }        },        "d3830d8299cb427f98d210867c40b03e": {          "model_module": "@jupyter-widgets/controls",          "model_name": "HTMLModel",          "model_module_version": "1.5.0",          "state": {            "_dom_classes": [],            "_model_module": "@jupyter-widgets/controls",            "_model_module_version": "1.5.0",            "_model_name": "HTMLModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/controls",            "_view_module_version": "1.5.0",            "_view_name": "HTMLView",            "description": "",            "description_tooltip": null,            "layout": "IPY_MODEL_3870cc0850184ac98dfe1fe646d19acc",            "placeholder": "​",            "style": "IPY_MODEL_18d59808f7ab4ff7a5a69fcfe245cb91",            "value": "sentence_bert_config.json: 100%"          }        },        "b17e87e86068463b81823634d795ad4d": {          "model_module": "@jupyter-widgets/controls",          "model_name": "FloatProgressModel",          "model_module_version": "1.5.0",          "state": {            "_dom_classes": [],            "_model_module": "@jupyter-widgets/controls",            "_model_module_version": "1.5.0",            "_model_name": "FloatProgressModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/controls",            "_view_module_version": "1.5.0",            "_view_name": "ProgressView",            "bar_style": "success",            "description": "",            "description_tooltip": null,            "layout": "IPY_MODEL_3faa24a0c6ad4138b19447a9cae1078a",            "max": 53,            "min": 0,            "orientation": "horizontal",            "style": "IPY_MODEL_a4e67a6f695448e884eb62395f1ab389",            "value": 53          }        },        "c23381aba2b54442840843488d2b3b34": {          "model_module": "@jupyter-widgets/controls",          "model_name": "HTMLModel",          "model_module_version": "1.5.0",          "state": {            "_dom_classes": [],            "_model_module": "@jupyter-widgets/controls",            "_model_module_version": "1.5.0",            "_model_name": "HTMLModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/controls",            "_view_module_version": "1.5.0",            "_view_name": "HTMLView",            "description": "",            "description_tooltip": null,            "layout": "IPY_MODEL_da00e3d12c1d420091d8ba701184e51c",            "placeholder": "​",            "style": "IPY_MODEL_db2c37fc93ba4118ba24517a5269d3b0",            "value": " 53.0/53.0 [00:00&lt;00:00, 5.59kB/s]"          }        },        "b8e1668a7dbc41939b6cdd1b7ed55050": {          "model_module": "@jupyter-widgets/base",          "model_name": "LayoutModel",          "model_module_version": "1.2.0",          "state": {            "_model_module": "@jupyter-widgets/base",            "_model_module_version": "1.2.0",            "_model_name": "LayoutModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/base",            "_view_module_version": "1.2.0",            "_view_name": "LayoutView",            "align_content": null,            "align_items": null,            "align_self": null,            "border": null,            "bottom": null,            "display": null,            "flex": null,            "flex_flow": null,            "grid_area": null,            "grid_auto_columns": null,            "grid_auto_flow": null,            "grid_auto_rows": null,            "grid_column": null,            "grid_gap": null,            "grid_row": null,            "grid_template_areas": null,            "grid_template_columns": null,            "grid_template_rows": null,            "height": null,            "justify_content": null,            "justify_items": null,            "left": null,            "margin": null,            "max_height": null,            "max_width": null,            "min_height": null,            "min_width": null,            "object_fit": null,            "object_position": null,            "order": null,            "overflow": null,            "overflow_x": null,            "overflow_y": null,            "padding": null,            "right": null,            "top": null,            "visibility": null,            "width": null          }        },        "3870cc0850184ac98dfe1fe646d19acc": {          "model_module": "@jupyter-widgets/base",          "model_name": "LayoutModel",          "model_module_version": "1.2.0",          "state": {            "_model_module": "@jupyter-widgets/base",            "_model_module_version": "1.2.0",            "_model_name": "LayoutModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/base",            "_view_module_version": "1.2.0",            "_view_name": "LayoutView",            "align_content": null,            "align_items": null,            "align_self": null,            "border": null,            "bottom": null,            "display": null,            "flex": null,            "flex_flow": null,            "grid_area": null,            "grid_auto_columns": null,            "grid_auto_flow": null,            "grid_auto_rows": null,            "grid_column": null,            "grid_gap": null,            "grid_row": null,            "grid_template_areas": null,            "grid_template_columns": null,            "grid_template_rows": null,            "height": null,            "justify_content": null,            "justify_items": null,            "left": null,            "margin": null,            "max_height": null,            "max_width": null,            "min_height": null,            "min_width": null,            "object_fit": null,            "object_position": null,            "order": null,            "overflow": null,            "overflow_x": null,            "overflow_y": null,            "padding": null,            "right": null,            "top": null,            "visibility": null,            "width": null          }        },        "18d59808f7ab4ff7a5a69fcfe245cb91": {          "model_module": "@jupyter-widgets/controls",          "model_name": "DescriptionStyleModel",          "model_module_version": "1.5.0",          "state": {            "_model_module": "@jupyter-widgets/controls",            "_model_module_version": "1.5.0",            "_model_name": "DescriptionStyleModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/base",            "_view_module_version": "1.2.0",            "_view_name": "StyleView",            "description_width": ""          }        },        "3faa24a0c6ad4138b19447a9cae1078a": {          "model_module": "@jupyter-widgets/base",          "model_name": "LayoutModel",          "model_module_version": "1.2.0",          "state": {            "_model_module": "@jupyter-widgets/base",            "_model_module_version": "1.2.0",            "_model_name": "LayoutModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/base",            "_view_module_version": "1.2.0",            "_view_name": "LayoutView",            "align_content": null,            "align_items": null,            "align_self": null,            "border": null,            "bottom": null,            "display": null,            "flex": null,            "flex_flow": null,            "grid_area": null,            "grid_auto_columns": null,            "grid_auto_flow": null,            "grid_auto_rows": null,            "grid_column": null,            "grid_gap": null,            "grid_row": null,            "grid_template_areas": null,            "grid_template_columns": null,            "grid_template_rows": null,            "height": null,            "justify_content": null,            "justify_items": null,            "left": null,            "margin": null,            "max_height": null,            "max_width": null,            "min_height": null,            "min_width": null,            "object_fit": null,            "object_position": null,            "order": null,            "overflow": null,            "overflow_x": null,            "overflow_y": null,            "padding": null,            "right": null,            "top": null,            "visibility": null,            "width": null          }        },        "a4e67a6f695448e884eb62395f1ab389": {          "model_module": "@jupyter-widgets/controls",          "model_name": "ProgressStyleModel",          "model_module_version": "1.5.0",          "state": {            "_model_module": "@jupyter-widgets/controls",            "_model_module_version": "1.5.0",            "_model_name": "ProgressStyleModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/base",            "_view_module_version": "1.2.0",            "_view_name": "StyleView",            "bar_color": null,            "description_width": ""          }        },        "da00e3d12c1d420091d8ba701184e51c": {          "model_module": "@jupyter-widgets/base",          "model_name": "LayoutModel",          "model_module_version": "1.2.0",          "state": {            "_model_module": "@jupyter-widgets/base",            "_model_module_version": "1.2.0",            "_model_name": "LayoutModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/base",            "_view_module_version": "1.2.0",            "_view_name": "LayoutView",            "align_content": null,            "align_items": null,            "align_self": null,            "border": null,            "bottom": null,            "display": null,            "flex": null,            "flex_flow": null,            "grid_area": null,            "grid_auto_columns": null,            "grid_auto_flow": null,            "grid_auto_rows": null,            "grid_column": null,            "grid_gap": null,            "grid_row": null,            "grid_template_areas": null,            "grid_template_columns": null,            "grid_template_rows": null,            "height": null,            "justify_content": null,            "justify_items": null,            "left": null,            "margin": null,            "max_height": null,            "max_width": null,            "min_height": null,            "min_width": null,            "object_fit": null,            "object_position": null,            "order": null,            "overflow": null,            "overflow_x": null,            "overflow_y": null,            "padding": null,            "right": null,            "top": null,            "visibility": null,            "width": null          }        },        "db2c37fc93ba4118ba24517a5269d3b0": {          "model_module": "@jupyter-widgets/controls",          "model_name": "DescriptionStyleModel",          "model_module_version": "1.5.0",          "state": {            "_model_module": "@jupyter-widgets/controls",            "_model_module_version": "1.5.0",            "_model_name": "DescriptionStyleModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/base",            "_view_module_version": "1.2.0",            "_view_name": "StyleView",            "description_width": ""          }        },        "cd3080bbee28450b98dc45d638ddcee2": {          "model_module": "@jupyter-widgets/controls",          "model_name": "HBoxModel",          "model_module_version": "1.5.0",          "state": {            "_dom_classes": [],            "_model_module": "@jupyter-widgets/controls",            "_model_module_version": "1.5.0",            "_model_name": "HBoxModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/controls",            "_view_module_version": "1.5.0",            "_view_name": "HBoxView",            "box_style": "",            "children": [              "IPY_MODEL_471f0d5c20474d0c92cd376dafc27a8a",              "IPY_MODEL_32980f43684f4322ae7f7b10be808d94",              "IPY_MODEL_98c67eb20001423e8e8ff6ee073409da"            ],            "layout": "IPY_MODEL_f5245838df4d4471b6c9b9659bce8147"          }        },        "471f0d5c20474d0c92cd376dafc27a8a": {          "model_module": "@jupyter-widgets/controls",          "model_name": "HTMLModel",          "model_module_version": "1.5.0",          "state": {            "_dom_classes": [],            "_model_module": "@jupyter-widgets/controls",            "_model_module_version": "1.5.0",            "_model_name": "HTMLModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/controls",            "_view_module_version": "1.5.0",            "_view_name": "HTMLView",            "description": "",            "description_tooltip": null,            "layout": "IPY_MODEL_a131a1ad4dd84fbf9a11e48500c5e7a8",            "placeholder": "​",            "style": "IPY_MODEL_c236f4bc518444b6b458cb6995eb04fa",            "value": "config.json: "          }        },        "32980f43684f4322ae7f7b10be808d94": {          "model_module": "@jupyter-widgets/controls",          "model_name": "FloatProgressModel",          "model_module_version": "1.5.0",          "state": {            "_dom_classes": [],            "_model_module": "@jupyter-widgets/controls",            "_model_module_version": "1.5.0",            "_model_name": "FloatProgressModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/controls",            "_view_module_version": "1.5.0",            "_view_name": "ProgressView",            "bar_style": "success",            "description": "",            "description_tooltip": null,            "layout": "IPY_MODEL_1d32301d8d3c438b989077f1f164f54b",            "max": 1,            "min": 0,            "orientation": "horizontal",            "style": "IPY_MODEL_8df0689088cb40518c97187d22585054",            "value": 1          }        },        "98c67eb20001423e8e8ff6ee073409da": {          "model_module": "@jupyter-widgets/controls",          "model_name": "HTMLModel",          "model_module_version": "1.5.0",          "state": {            "_dom_classes": [],            "_model_module": "@jupyter-widgets/controls",            "_model_module_version": "1.5.0",            "_model_name": "HTMLModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/controls",            "_view_module_version": "1.5.0",            "_view_name": "HTMLView",            "description": "",            "description_tooltip": null,            "layout": "IPY_MODEL_82f91f0fd4fe4af4928b016ba2c6344d",            "placeholder": "​",            "style": "IPY_MODEL_ff0d41c19f0440debf3aaad60c433877",            "value": " 1.52k/? [00:00&lt;00:00, 79.1kB/s]"          }        },        "f5245838df4d4471b6c9b9659bce8147": {          "model_module": "@jupyter-widgets/base",          "model_name": "LayoutModel",          "model_module_version": "1.2.0",          "state": {            "_model_module": "@jupyter-widgets/base",            "_model_module_version": "1.2.0",            "_model_name": "LayoutModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/base",            "_view_module_version": "1.2.0",            "_view_name": "LayoutView",            "align_content": null,            "align_items": null,            "align_self": null,            "border": null,            "bottom": null,            "display": null,            "flex": null,            "flex_flow": null,            "grid_area": null,            "grid_auto_columns": null,            "grid_auto_flow": null,            "grid_auto_rows": null,            "grid_column": null,            "grid_gap": null,            "grid_row": null,            "grid_template_areas": null,            "grid_template_columns": null,            "grid_template_rows": null,            "height": null,            "justify_content": null,            "justify_items": null,            "left": null,            "margin": null,            "max_height": null,            "max_width": null,            "min_height": null,            "min_width": null,            "object_fit": null,            "object_position": null,            "order": null,            "overflow": null,            "overflow_x": null,            "overflow_y": null,            "padding": null,            "right": null,            "top": null,            "visibility": null,            "width": null          }        },        "a131a1ad4dd84fbf9a11e48500c5e7a8": {          "model_module": "@jupyter-widgets/base",          "model_name": "LayoutModel",          "model_module_version": "1.2.0",          "state": {            "_model_module": "@jupyter-widgets/base",            "_model_module_version": "1.2.0",            "_model_name": "LayoutModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/base",            "_view_module_version": "1.2.0",            "_view_name": "LayoutView",            "align_content": null,            "align_items": null,            "align_self": null,            "border": null,            "bottom": null,            "display": null,            "flex": null,            "flex_flow": null,            "grid_area": null,            "grid_auto_columns": null,            "grid_auto_flow": null,            "grid_auto_rows": null,            "grid_column": null,            "grid_gap": null,            "grid_row": null,            "grid_template_areas": null,            "grid_template_columns": null,            "grid_template_rows": null,            "height": null,            "justify_content": null,            "justify_items": null,            "left": null,            "margin": null,            "max_height": null,            "max_width": null,            "min_height": null,            "min_width": null,            "object_fit": null,            "object_position": null,            "order": null,            "overflow": null,            "overflow_x": null,            "overflow_y": null,            "padding": null,            "right": null,            "top": null,            "visibility": null,            "width": null          }        },        "c236f4bc518444b6b458cb6995eb04fa": {          "model_module": "@jupyter-widgets/controls",          "model_name": "DescriptionStyleModel",          "model_module_version": "1.5.0",          "state": {            "_model_module": "@jupyter-widgets/controls",            "_model_module_version": "1.5.0",            "_model_name": "DescriptionStyleModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/base",            "_view_module_version": "1.2.0",            "_view_name": "StyleView",            "description_width": ""          }        },        "1d32301d8d3c438b989077f1f164f54b": {          "model_module": "@jupyter-widgets/base",          "model_name": "LayoutModel",          "model_module_version": "1.2.0",          "state": {            "_model_module": "@jupyter-widgets/base",            "_model_module_version": "1.2.0",            "_model_name": "LayoutModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/base",            "_view_module_version": "1.2.0",            "_view_name": "LayoutView",            "align_content": null,            "align_items": null,            "align_self": null,            "border": null,            "bottom": null,            "display": null,            "flex": null,            "flex_flow": null,            "grid_area": null,            "grid_auto_columns": null,            "grid_auto_flow": null,            "grid_auto_rows": null,            "grid_column": null,            "grid_gap": null,            "grid_row": null,            "grid_template_areas": null,            "grid_template_columns": null,            "grid_template_rows": null,            "height": null,            "justify_content": null,            "justify_items": null,            "left": null,            "margin": null,            "max_height": null,            "max_width": null,            "min_height": null,            "min_width": null,            "object_fit": null,            "object_position": null,            "order": null,            "overflow": null,            "overflow_x": null,            "overflow_y": null,            "padding": null,            "right": null,            "top": null,            "visibility": null,            "width": "20px"          }        },        "8df0689088cb40518c97187d22585054": {          "model_module": "@jupyter-widgets/controls",          "model_name": "ProgressStyleModel",          "model_module_version": "1.5.0",          "state": {            "_model_module": "@jupyter-widgets/controls",            "_model_module_version": "1.5.0",            "_model_name": "ProgressStyleModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/base",            "_view_module_version": "1.2.0",            "_view_name": "StyleView",            "bar_color": null,            "description_width": ""          }        },        "82f91f0fd4fe4af4928b016ba2c6344d": {          "model_module": "@jupyter-widgets/base",          "model_name": "LayoutModel",          "model_module_version": "1.2.0",          "state": {            "_model_module": "@jupyter-widgets/base",            "_model_module_version": "1.2.0",            "_model_name": "LayoutModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/base",            "_view_module_version": "1.2.0",            "_view_name": "LayoutView",            "align_content": null,            "align_items": null,            "align_self": null,            "border": null,            "bottom": null,            "display": null,            "flex": null,            "flex_flow": null,            "grid_area": null,            "grid_auto_columns": null,            "grid_auto_flow": null,            "grid_auto_rows": null,            "grid_column": null,            "grid_gap": null,            "grid_row": null,            "grid_template_areas": null,            "grid_template_columns": null,            "grid_template_rows": null,            "height": null,            "justify_content": null,            "justify_items": null,            "left": null,            "margin": null,            "max_height": null,            "max_width": null,            "min_height": null,            "min_width": null,            "object_fit": null,            "object_position": null,            "order": null,            "overflow": null,            "overflow_x": null,            "overflow_y": null,            "padding": null,            "right": null,            "top": null,            "visibility": null,            "width": null          }        },        "ff0d41c19f0440debf3aaad60c433877": {          "model_module": "@jupyter-widgets/controls",          "model_name": "DescriptionStyleModel",          "model_module_version": "1.5.0",          "state": {            "_model_module": "@jupyter-widgets/controls",            "_model_module_version": "1.5.0",            "_model_name": "DescriptionStyleModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/base",            "_view_module_version": "1.2.0",            "_view_name": "StyleView",            "description_width": ""          }        },        "3f5b915dff9c4c3f9676d4009da962ad": {          "model_module": "@jupyter-widgets/controls",          "model_name": "HBoxModel",          "model_module_version": "1.5.0",          "state": {            "_dom_classes": [],            "_model_module": "@jupyter-widgets/controls",            "_model_module_version": "1.5.0",            "_model_name": "HBoxModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/controls",            "_view_module_version": "1.5.0",            "_view_name": "HBoxView",            "box_style": "",            "children": [              "IPY_MODEL_112c143b55a34f9b9aed31679dd66d6b",              "IPY_MODEL_fcf1c33e86654e2386f6960ee71adf4a",              "IPY_MODEL_298d6134df294737aaf49bb1fd0eff6b"            ],            "layout": "IPY_MODEL_ac987e8442ef41488f706b220ab89a67"          }        },        "112c143b55a34f9b9aed31679dd66d6b": {          "model_module": "@jupyter-widgets/controls",          "model_name": "HTMLModel",          "model_module_version": "1.5.0",          "state": {            "_dom_classes": [],            "_model_module": "@jupyter-widgets/controls",            "_model_module_version": "1.5.0",            "_model_name": "HTMLModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/controls",            "_view_module_version": "1.5.0",            "_view_name": "HTMLView",            "description": "",            "description_tooltip": null,            "layout": "IPY_MODEL_19faa389154a4fdf8d04c1458401655f",            "placeholder": "​",            "style": "IPY_MODEL_4a1c6d3c532b427ebabd3d6fce842ad3",            "value": "pytorch_model.bin: 100%"          }        },        "fcf1c33e86654e2386f6960ee71adf4a": {          "model_module": "@jupyter-widgets/controls",          "model_name": "FloatProgressModel",          "model_module_version": "1.5.0",          "state": {            "_dom_classes": [],            "_model_module": "@jupyter-widgets/controls",            "_model_module_version": "1.5.0",            "_model_name": "FloatProgressModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/controls",            "_view_module_version": "1.5.0",            "_view_name": "ProgressView",            "bar_style": "success",            "description": "",            "description_tooltip": null,            "layout": "IPY_MODEL_2ff05460fad440a3827b0d7f84f1ff81",            "max": 4963705019,            "min": 0,            "orientation": "horizontal",            "style": "IPY_MODEL_1f99d6b0fb1d4ec8b43d9ab9063875b1",            "value": 4963705019          }        },        "298d6134df294737aaf49bb1fd0eff6b": {          "model_module": "@jupyter-widgets/controls",          "model_name": "HTMLModel",          "model_module_version": "1.5.0",          "state": {            "_dom_classes": [],            "_model_module": "@jupyter-widgets/controls",            "_model_module_version": "1.5.0",            "_model_name": "HTMLModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/controls",            "_view_module_version": "1.5.0",            "_view_name": "HTMLView",            "description": "",            "description_tooltip": null,            "layout": "IPY_MODEL_6fe5f5ec79e34c27895fe5cac6489cb7",            "placeholder": "​",            "style": "IPY_MODEL_3307bde944a541aaae3bbcb14a369675",            "value": " 4.96G/4.96G [00:24&lt;00:00, 210MB/s]"          }        },        "ac987e8442ef41488f706b220ab89a67": {          "model_module": "@jupyter-widgets/base",          "model_name": "LayoutModel",          "model_module_version": "1.2.0",          "state": {            "_model_module": "@jupyter-widgets/base",            "_model_module_version": "1.2.0",            "_model_name": "LayoutModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/base",            "_view_module_version": "1.2.0",            "_view_name": "LayoutView",            "align_content": null,            "align_items": null,            "align_self": null,            "border": null,            "bottom": null,            "display": null,            "flex": null,            "flex_flow": null,            "grid_area": null,            "grid_auto_columns": null,            "grid_auto_flow": null,            "grid_auto_rows": null,            "grid_column": null,            "grid_gap": null,            "grid_row": null,            "grid_template_areas": null,            "grid_template_columns": null,            "grid_template_rows": null,            "height": null,            "justify_content": null,            "justify_items": null,            "left": null,            "margin": null,            "max_height": null,            "max_width": null,            "min_height": null,            "min_width": null,            "object_fit": null,            "object_position": null,            "order": null,            "overflow": null,            "overflow_x": null,            "overflow_y": null,            "padding": null,            "right": null,            "top": null,            "visibility": null,            "width": null          }        },        "19faa389154a4fdf8d04c1458401655f": {          "model_module": "@jupyter-widgets/base",          "model_name": "LayoutModel",          "model_module_version": "1.2.0",          "state": {            "_model_module": "@jupyter-widgets/base",            "_model_module_version": "1.2.0",            "_model_name": "LayoutModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/base",            "_view_module_version": "1.2.0",            "_view_name": "LayoutView",            "align_content": null,            "align_items": null,            "align_self": null,            "border": null,            "bottom": null,            "display": null,            "flex": null,            "flex_flow": null,            "grid_area": null,            "grid_auto_columns": null,            "grid_auto_flow": null,            "grid_auto_rows": null,            "grid_column": null,            "grid_gap": null,            "grid_row": null,            "grid_template_areas": null,            "grid_template_columns": null,            "grid_template_rows": null,            "height": null,            "justify_content": null,            "justify_items": null,            "left": null,            "margin": null,            "max_height": null,            "max_width": null,            "min_height": null,            "min_width": null,            "object_fit": null,            "object_position": null,            "order": null,            "overflow": null,            "overflow_x": null,            "overflow_y": null,            "padding": null,            "right": null,            "top": null,            "visibility": null,            "width": null          }        },        "4a1c6d3c532b427ebabd3d6fce842ad3": {          "model_module": "@jupyter-widgets/controls",          "model_name": "DescriptionStyleModel",          "model_module_version": "1.5.0",          "state": {            "_model_module": "@jupyter-widgets/controls",            "_model_module_version": "1.5.0",            "_model_name": "DescriptionStyleModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/base",            "_view_module_version": "1.2.0",            "_view_name": "StyleView",            "description_width": ""          }        },        "2ff05460fad440a3827b0d7f84f1ff81": {          "model_module": "@jupyter-widgets/base",          "model_name": "LayoutModel",          "model_module_version": "1.2.0",          "state": {            "_model_module": "@jupyter-widgets/base",            "_model_module_version": "1.2.0",            "_model_name": "LayoutModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/base",            "_view_module_version": "1.2.0",            "_view_name": "LayoutView",            "align_content": null,            "align_items": null,            "align_self": null,            "border": null,            "bottom": null,            "display": null,            "flex": null,            "flex_flow": null,            "grid_area": null,            "grid_auto_columns": null,            "grid_auto_flow": null,            "grid_auto_rows": null,            "grid_column": null,            "grid_gap": null,            "grid_row": null,            "grid_template_areas": null,            "grid_template_columns": null,            "grid_template_rows": null,            "height": null,            "justify_content": null,            "justify_items": null,            "left": null,            "margin": null,            "max_height": null,            "max_width": null,            "min_height": null,            "min_width": null,            "object_fit": null,            "object_position": null,            "order": null,            "overflow": null,            "overflow_x": null,            "overflow_y": null,            "padding": null,            "right": null,            "top": null,            "visibility": null,            "width": null          }        },        "1f99d6b0fb1d4ec8b43d9ab9063875b1": {          "model_module": "@jupyter-widgets/controls",          "model_name": "ProgressStyleModel",          "model_module_version": "1.5.0",          "state": {            "_model_module": "@jupyter-widgets/controls",            "_model_module_version": "1.5.0",            "_model_name": "ProgressStyleModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/base",            "_view_module_version": "1.2.0",            "_view_name": "StyleView",            "bar_color": null,            "description_width": ""          }        },        "6fe5f5ec79e34c27895fe5cac6489cb7": {          "model_module": "@jupyter-widgets/base",          "model_name": "LayoutModel",          "model_module_version": "1.2.0",          "state": {            "_model_module": "@jupyter-widgets/base",            "_model_module_version": "1.2.0",            "_model_name": "LayoutModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/base",            "_view_module_version": "1.2.0",            "_view_name": "LayoutView",            "align_content": null,            "align_items": null,            "align_self": null,            "border": null,            "bottom": null,            "display": null,            "flex": null,            "flex_flow": null,            "grid_area": null,            "grid_auto_columns": null,            "grid_auto_flow": null,            "grid_auto_rows": null,            "grid_column": null,            "grid_gap": null,            "grid_row": null,            "grid_template_areas": null,            "grid_template_columns": null,            "grid_template_rows": null,            "height": null,            "justify_content": null,            "justify_items": null,            "left": null,            "margin": null,            "max_height": null,            "max_width": null,            "min_height": null,            "min_width": null,            "object_fit": null,            "object_position": null,            "order": null,            "overflow": null,            "overflow_x": null,            "overflow_y": null,            "padding": null,            "right": null,            "top": null,            "visibility": null,            "width": null          }        },        "3307bde944a541aaae3bbcb14a369675": {          "model_module": "@jupyter-widgets/controls",          "model_name": "DescriptionStyleModel",          "model_module_version": "1.5.0",          "state": {            "_model_module": "@jupyter-widgets/controls",            "_model_module_version": "1.5.0",            "_model_name": "DescriptionStyleModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/base",            "_view_module_version": "1.2.0",            "_view_name": "StyleView",            "description_width": ""          }        },        "58bbeaf2cfca4a79b01aa6f7ac5e8932": {          "model_module": "@jupyter-widgets/controls",          "model_name": "HBoxModel",          "model_module_version": "1.5.0",          "state": {            "_dom_classes": [],            "_model_module": "@jupyter-widgets/controls",            "_model_module_version": "1.5.0",            "_model_name": "HBoxModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/controls",            "_view_module_version": "1.5.0",            "_view_name": "HBoxView",            "box_style": "",            "children": [              "IPY_MODEL_96eb85549d8641a28ea82e680b9f8964",              "IPY_MODEL_61b7d2c5d76f4dd48d1bf091998082a1",              "IPY_MODEL_a0d067955126420db4af9faf90184979"            ],            "layout": "IPY_MODEL_439df49e298b43618008f7198892aca6"          }        },        "96eb85549d8641a28ea82e680b9f8964": {          "model_module": "@jupyter-widgets/controls",          "model_name": "HTMLModel",          "model_module_version": "1.5.0",          "state": {            "_dom_classes": [],            "_model_module": "@jupyter-widgets/controls",            "_model_module_version": "1.5.0",            "_model_name": "HTMLModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/controls",            "_view_module_version": "1.5.0",            "_view_name": "HTMLView",            "description": "",            "description_tooltip": null,            "layout": "IPY_MODEL_438b614ece2c4f8cb177dd9ed487a3fe",            "placeholder": "​",            "style": "IPY_MODEL_e672670fd7904add8e20f35aa1aa7cd0",            "value": "model.safetensors: 100%"          }        },        "61b7d2c5d76f4dd48d1bf091998082a1": {          "model_module": "@jupyter-widgets/controls",          "model_name": "FloatProgressModel",          "model_module_version": "1.5.0",          "state": {            "_dom_classes": [],            "_model_module": "@jupyter-widgets/controls",            "_model_module_version": "1.5.0",            "_model_name": "FloatProgressModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/controls",            "_view_module_version": "1.5.0",            "_view_name": "ProgressView",            "bar_style": "success",            "description": "",            "description_tooltip": null,            "layout": "IPY_MODEL_0d737cffa94d464db9e9d17b434bd2df",            "max": 4963663166,            "min": 0,            "orientation": "horizontal",            "style": "IPY_MODEL_44c2f266274647edb4c0a1dcc744d2ae",            "value": 4963663166          }        },        "a0d067955126420db4af9faf90184979": {          "model_module": "@jupyter-widgets/controls",          "model_name": "HTMLModel",          "model_module_version": "1.5.0",          "state": {            "_dom_classes": [],            "_model_module": "@jupyter-widgets/controls",            "_model_module_version": "1.5.0",            "_model_name": "HTMLModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/controls",            "_view_module_version": "1.5.0",            "_view_name": "HTMLView",            "description": "",            "description_tooltip": null,            "layout": "IPY_MODEL_78a910d28070429f8769a664688905be",            "placeholder": "​",            "style": "IPY_MODEL_44bd5016067d42a19f6137200c3fa8ac",            "value": " 4.96G/4.96G [00:39&lt;00:00, 157MB/s]"          }        },        "439df49e298b43618008f7198892aca6": {          "model_module": "@jupyter-widgets/base",          "model_name": "LayoutModel",          "model_module_version": "1.2.0",          "state": {            "_model_module": "@jupyter-widgets/base",            "_model_module_version": "1.2.0",            "_model_name": "LayoutModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/base",            "_view_module_version": "1.2.0",            "_view_name": "LayoutView",            "align_content": null,            "align_items": null,            "align_self": null,            "border": null,            "bottom": null,            "display": null,            "flex": null,            "flex_flow": null,            "grid_area": null,            "grid_auto_columns": null,            "grid_auto_flow": null,            "grid_auto_rows": null,            "grid_column": null,            "grid_gap": null,            "grid_row": null,            "grid_template_areas": null,            "grid_template_columns": null,            "grid_template_rows": null,            "height": null,            "justify_content": null,            "justify_items": null,            "left": null,            "margin": null,            "max_height": null,            "max_width": null,            "min_height": null,            "min_width": null,            "object_fit": null,            "object_position": null,            "order": null,            "overflow": null,            "overflow_x": null,            "overflow_y": null,            "padding": null,            "right": null,            "top": null,            "visibility": null,            "width": null          }        },        "438b614ece2c4f8cb177dd9ed487a3fe": {          "model_module": "@jupyter-widgets/base",          "model_name": "LayoutModel",          "model_module_version": "1.2.0",          "state": {            "_model_module": "@jupyter-widgets/base",            "_model_module_version": "1.2.0",            "_model_name": "LayoutModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/base",            "_view_module_version": "1.2.0",            "_view_name": "LayoutView",            "align_content": null,            "align_items": null,            "align_self": null,            "border": null,            "bottom": null,            "display": null,            "flex": null,            "flex_flow": null,            "grid_area": null,            "grid_auto_columns": null,            "grid_auto_flow": null,            "grid_auto_rows": null,            "grid_column": null,            "grid_gap": null,            "grid_row": null,            "grid_template_areas": null,            "grid_template_columns": null,            "grid_template_rows": null,            "height": null,            "justify_content": null,            "justify_items": null,            "left": null,            "margin": null,            "max_height": null,            "max_width": null,            "min_height": null,            "min_width": null,            "object_fit": null,            "object_position": null,            "order": null,            "overflow": null,            "overflow_x": null,            "overflow_y": null,            "padding": null,            "right": null,            "top": null,            "visibility": null,            "width": null          }        },        "e672670fd7904add8e20f35aa1aa7cd0": {          "model_module": "@jupyter-widgets/controls",          "model_name": "DescriptionStyleModel",          "model_module_version": "1.5.0",          "state": {            "_model_module": "@jupyter-widgets/controls",            "_model_module_version": "1.5.0",            "_model_name": "DescriptionStyleModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/base",            "_view_module_version": "1.2.0",            "_view_name": "StyleView",            "description_width": ""          }        },        "0d737cffa94d464db9e9d17b434bd2df": {          "model_module": "@jupyter-widgets/base",          "model_name": "LayoutModel",          "model_module_version": "1.2.0",          "state": {            "_model_module": "@jupyter-widgets/base",            "_model_module_version": "1.2.0",            "_model_name": "LayoutModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/base",            "_view_module_version": "1.2.0",            "_view_name": "LayoutView",            "align_content": null,            "align_items": null,            "align_self": null,            "border": null,            "bottom": null,            "display": null,            "flex": null,            "flex_flow": null,            "grid_area": null,            "grid_auto_columns": null,            "grid_auto_flow": null,            "grid_auto_rows": null,            "grid_column": null,            "grid_gap": null,            "grid_row": null,            "grid_template_areas": null,            "grid_template_columns": null,            "grid_template_rows": null,            "height": null,            "justify_content": null,            "justify_items": null,            "left": null,            "margin": null,            "max_height": null,            "max_width": null,            "min_height": null,            "min_width": null,            "object_fit": null,            "object_position": null,            "order": null,            "overflow": null,            "overflow_x": null,            "overflow_y": null,            "padding": null,            "right": null,            "top": null,            "visibility": null,            "width": null          }        },        "44c2f266274647edb4c0a1dcc744d2ae": {          "model_module": "@jupyter-widgets/controls",          "model_name": "ProgressStyleModel",          "model_module_version": "1.5.0",          "state": {            "_model_module": "@jupyter-widgets/controls",            "_model_module_version": "1.5.0",            "_model_name": "ProgressStyleModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/base",            "_view_module_version": "1.2.0",            "_view_name": "StyleView",            "bar_color": null,            "description_width": ""          }        },        "78a910d28070429f8769a664688905be": {          "model_module": "@jupyter-widgets/base",          "model_name": "LayoutModel",          "model_module_version": "1.2.0",          "state": {            "_model_module": "@jupyter-widgets/base",            "_model_module_version": "1.2.0",            "_model_name": "LayoutModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/base",            "_view_module_version": "1.2.0",            "_view_name": "LayoutView",            "align_content": null,            "align_items": null,            "align_self": null,            "border": null,            "bottom": null,            "display": null,            "flex": null,            "flex_flow": null,            "grid_area": null,            "grid_auto_columns": null,            "grid_auto_flow": null,            "grid_auto_rows": null,            "grid_column": null,            "grid_gap": null,            "grid_row": null,            "grid_template_areas": null,            "grid_template_columns": null,            "grid_template_rows": null,            "height": null,            "justify_content": null,            "justify_items": null,            "left": null,            "margin": null,            "max_height": null,            "max_width": null,            "min_height": null,            "min_width": null,            "object_fit": null,            "object_position": null,            "order": null,            "overflow": null,            "overflow_x": null,            "overflow_y": null,            "padding": null,            "right": null,            "top": null,            "visibility": null,            "width": null          }        },        "44bd5016067d42a19f6137200c3fa8ac": {          "model_module": "@jupyter-widgets/controls",          "model_name": "DescriptionStyleModel",          "model_module_version": "1.5.0",          "state": {            "_model_module": "@jupyter-widgets/controls",            "_model_module_version": "1.5.0",            "_model_name": "DescriptionStyleModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/base",            "_view_module_version": "1.2.0",            "_view_name": "StyleView",            "description_width": ""          }        },        "07eae94ddf2745129a3c63d0d3d66f27": {          "model_module": "@jupyter-widgets/controls",          "model_name": "HBoxModel",          "model_module_version": "1.5.0",          "state": {            "_dom_classes": [],            "_model_module": "@jupyter-widgets/controls",            "_model_module_version": "1.5.0",            "_model_name": "HBoxModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/controls",            "_view_module_version": "1.5.0",            "_view_name": "HBoxView",            "box_style": "",            "children": [              "IPY_MODEL_7b9df8eb955f4bb7b4c4417bc959faa6",              "IPY_MODEL_99a179cdf26c4c3c8f08e8af732e7f5c",              "IPY_MODEL_fbc432f9254b48e4a01fcd3d618b2a17"            ],            "layout": "IPY_MODEL_ea385f44d0094416b5a26727da8fd7bc"          }        },        "7b9df8eb955f4bb7b4c4417bc959faa6": {          "model_module": "@jupyter-widgets/controls",          "model_name": "HTMLModel",          "model_module_version": "1.5.0",          "state": {            "_dom_classes": [],            "_model_module": "@jupyter-widgets/controls",            "_model_module_version": "1.5.0",            "_model_name": "HTMLModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/controls",            "_view_module_version": "1.5.0",            "_view_name": "HTMLView",            "description": "",            "description_tooltip": null,            "layout": "IPY_MODEL_c0ff09d6a4164de8bcc66544527ff79c",            "placeholder": "​",            "style": "IPY_MODEL_d6ecc886777d499e8652ec1d042229a4",            "value": "tokenizer_config.json: "          }        },        "99a179cdf26c4c3c8f08e8af732e7f5c": {          "model_module": "@jupyter-widgets/controls",          "model_name": "FloatProgressModel",          "model_module_version": "1.5.0",          "state": {            "_dom_classes": [],            "_model_module": "@jupyter-widgets/controls",            "_model_module_version": "1.5.0",            "_model_name": "FloatProgressModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/controls",            "_view_module_version": "1.5.0",            "_view_name": "ProgressView",            "bar_style": "success",            "description": "",            "description_tooltip": null,            "layout": "IPY_MODEL_9f751a279f8a42e4bcbdd3a8009118da",            "max": 1,            "min": 0,            "orientation": "horizontal",            "style": "IPY_MODEL_7a6e795cd6c9425d9f0cebbbcef65b24",            "value": 1          }        },        "fbc432f9254b48e4a01fcd3d618b2a17": {          "model_module": "@jupyter-widgets/controls",          "model_name": "HTMLModel",          "model_module_version": "1.5.0",          "state": {            "_dom_classes": [],            "_model_module": "@jupyter-widgets/controls",            "_model_module_version": "1.5.0",            "_model_name": "HTMLModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/controls",            "_view_module_version": "1.5.0",            "_view_name": "HTMLView",            "description": "",            "description_tooltip": null,            "layout": "IPY_MODEL_73528a41ad52491abccdbd3b93f840ba",            "placeholder": "​",            "style": "IPY_MODEL_95c884653bf94de7a7adba915f1a76ab",            "value": " 2.40k/? [00:00&lt;00:00, 91.3kB/s]"          }        },        "ea385f44d0094416b5a26727da8fd7bc": {          "model_module": "@jupyter-widgets/base",          "model_name": "LayoutModel",          "model_module_version": "1.2.0",          "state": {            "_model_module": "@jupyter-widgets/base",            "_model_module_version": "1.2.0",            "_model_name": "LayoutModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/base",            "_view_module_version": "1.2.0",            "_view_name": "LayoutView",            "align_content": null,            "align_items": null,            "align_self": null,            "border": null,            "bottom": null,            "display": null,            "flex": null,            "flex_flow": null,            "grid_area": null,            "grid_auto_columns": null,            "grid_auto_flow": null,            "grid_auto_rows": null,            "grid_column": null,            "grid_gap": null,            "grid_row": null,            "grid_template_areas": null,            "grid_template_columns": null,            "grid_template_rows": null,            "height": null,            "justify_content": null,            "justify_items": null,            "left": null,            "margin": null,            "max_height": null,            "max_width": null,            "min_height": null,            "min_width": null,            "object_fit": null,            "object_position": null,            "order": null,            "overflow": null,            "overflow_x": null,            "overflow_y": null,            "padding": null,            "right": null,            "top": null,            "visibility": null,            "width": null          }        },        "c0ff09d6a4164de8bcc66544527ff79c": {          "model_module": "@jupyter-widgets/base",          "model_name": "LayoutModel",          "model_module_version": "1.2.0",          "state": {            "_model_module": "@jupyter-widgets/base",            "_model_module_version": "1.2.0",            "_model_name": "LayoutModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/base",            "_view_module_version": "1.2.0",            "_view_name": "LayoutView",            "align_content": null,            "align_items": null,            "align_self": null,            "border": null,            "bottom": null,            "display": null,            "flex": null,            "flex_flow": null,            "grid_area": null,            "grid_auto_columns": null,            "grid_auto_flow": null,            "grid_auto_rows": null,            "grid_column": null,            "grid_gap": null,            "grid_row": null,            "grid_template_areas": null,            "grid_template_columns": null,            "grid_template_rows": null,            "height": null,            "justify_content": null,            "justify_items": null,            "left": null,            "margin": null,            "max_height": null,            "max_width": null,            "min_height": null,            "min_width": null,            "object_fit": null,            "object_position": null,            "order": null,            "overflow": null,            "overflow_x": null,            "overflow_y": null,            "padding": null,            "right": null,            "top": null,            "visibility": null,            "width": null          }        },        "d6ecc886777d499e8652ec1d042229a4": {          "model_module": "@jupyter-widgets/controls",          "model_name": "DescriptionStyleModel",          "model_module_version": "1.5.0",          "state": {            "_model_module": "@jupyter-widgets/controls",            "_model_module_version": "1.5.0",            "_model_name": "DescriptionStyleModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/base",            "_view_module_version": "1.2.0",            "_view_name": "StyleView",            "description_width": ""          }        },        "9f751a279f8a42e4bcbdd3a8009118da": {          "model_module": "@jupyter-widgets/base",          "model_name": "LayoutModel",          "model_module_version": "1.2.0",          "state": {            "_model_module": "@jupyter-widgets/base",            "_model_module_version": "1.2.0",            "_model_name": "LayoutModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/base",            "_view_module_version": "1.2.0",            "_view_name": "LayoutView",            "align_content": null,            "align_items": null,            "align_self": null,            "border": null,            "bottom": null,            "display": null,            "flex": null,            "flex_flow": null,            "grid_area": null,            "grid_auto_columns": null,            "grid_auto_flow": null,            "grid_auto_rows": null,            "grid_column": null,            "grid_gap": null,            "grid_row": null,            "grid_template_areas": null,            "grid_template_columns": null,            "grid_template_rows": null,            "height": null,            "justify_content": null,            "justify_items": null,            "left": null,            "margin": null,            "max_height": null,            "max_width": null,            "min_height": null,            "min_width": null,            "object_fit": null,            "object_position": null,            "order": null,            "overflow": null,            "overflow_x": null,            "overflow_y": null,            "padding": null,            "right": null,            "top": null,            "visibility": null,            "width": "20px"          }        },        "7a6e795cd6c9425d9f0cebbbcef65b24": {          "model_module": "@jupyter-widgets/controls",          "model_name": "ProgressStyleModel",          "model_module_version": "1.5.0",          "state": {            "_model_module": "@jupyter-widgets/controls",            "_model_module_version": "1.5.0",            "_model_name": "ProgressStyleModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/base",            "_view_module_version": "1.2.0",            "_view_name": "StyleView",            "bar_color": null,            "description_width": ""          }        },        "73528a41ad52491abccdbd3b93f840ba": {          "model_module": "@jupyter-widgets/base",          "model_name": "LayoutModel",          "model_module_version": "1.2.0",          "state": {            "_model_module": "@jupyter-widgets/base",            "_model_module_version": "1.2.0",            "_model_name": "LayoutModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/base",            "_view_module_version": "1.2.0",            "_view_name": "LayoutView",            "align_content": null,            "align_items": null,            "align_self": null,            "border": null,            "bottom": null,            "display": null,            "flex": null,            "flex_flow": null,            "grid_area": null,            "grid_auto_columns": null,            "grid_auto_flow": null,            "grid_auto_rows": null,            "grid_column": null,            "grid_gap": null,            "grid_row": null,            "grid_template_areas": null,            "grid_template_columns": null,            "grid_template_rows": null,            "height": null,            "justify_content": null,            "justify_items": null,            "left": null,            "margin": null,            "max_height": null,            "max_width": null,            "min_height": null,            "min_width": null,            "object_fit": null,            "object_position": null,            "order": null,            "overflow": null,            "overflow_x": null,            "overflow_y": null,            "padding": null,            "right": null,            "top": null,            "visibility": null,            "width": null          }        },        "95c884653bf94de7a7adba915f1a76ab": {          "model_module": "@jupyter-widgets/controls",          "model_name": "DescriptionStyleModel",          "model_module_version": "1.5.0",          "state": {            "_model_module": "@jupyter-widgets/controls",            "_model_module_version": "1.5.0",            "_model_name": "DescriptionStyleModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/base",            "_view_module_version": "1.2.0",            "_view_name": "StyleView",            "description_width": ""          }        },        "10f9babd755e4ca0bb36175c34321bee": {          "model_module": "@jupyter-widgets/controls",          "model_name": "HBoxModel",          "model_module_version": "1.5.0",          "state": {            "_dom_classes": [],            "_model_module": "@jupyter-widgets/controls",            "_model_module_version": "1.5.0",            "_model_name": "HBoxModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/controls",            "_view_module_version": "1.5.0",            "_view_name": "HBoxView",            "box_style": "",            "children": [              "IPY_MODEL_80286c74df584858af8862bc283b32f3",              "IPY_MODEL_ad94fa23988243eea29d31e3ab816773",              "IPY_MODEL_567e300b6ab74ab0b8d793e8c6559483"            ],            "layout": "IPY_MODEL_e5a556ff241c4621982b2b8d57e295a7"          }        },        "80286c74df584858af8862bc283b32f3": {          "model_module": "@jupyter-widgets/controls",          "model_name": "HTMLModel",          "model_module_version": "1.5.0",          "state": {            "_dom_classes": [],            "_model_module": "@jupyter-widgets/controls",            "_model_module_version": "1.5.0",            "_model_name": "HTMLModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/controls",            "_view_module_version": "1.5.0",            "_view_name": "HTMLView",            "description": "",            "description_tooltip": null,            "layout": "IPY_MODEL_e2ca1a1eea0442819f120b66dcd83053",            "placeholder": "​",            "style": "IPY_MODEL_cc7c897f01cd45338197b4680898a70c",            "value": "spiece.model: 100%"          }        },        "ad94fa23988243eea29d31e3ab816773": {          "model_module": "@jupyter-widgets/controls",          "model_name": "FloatProgressModel",          "model_module_version": "1.5.0",          "state": {            "_dom_classes": [],            "_model_module": "@jupyter-widgets/controls",            "_model_module_version": "1.5.0",            "_model_name": "FloatProgressModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/controls",            "_view_module_version": "1.5.0",            "_view_name": "ProgressView",            "bar_style": "success",            "description": "",            "description_tooltip": null,            "layout": "IPY_MODEL_4c9e9052f056489d9ae3917ea04a1634",            "max": 791656,            "min": 0,            "orientation": "horizontal",            "style": "IPY_MODEL_549cff618a2a4a0182fe9f4401619207",            "value": 791656          }        },        "567e300b6ab74ab0b8d793e8c6559483": {          "model_module": "@jupyter-widgets/controls",          "model_name": "HTMLModel",          "model_module_version": "1.5.0",          "state": {            "_dom_classes": [],            "_model_module": "@jupyter-widgets/controls",            "_model_module_version": "1.5.0",            "_model_name": "HTMLModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/controls",            "_view_module_version": "1.5.0",            "_view_name": "HTMLView",            "description": "",            "description_tooltip": null,            "layout": "IPY_MODEL_064b297dac5c4067a987bdfdc03988bc",            "placeholder": "​",            "style": "IPY_MODEL_b660155270d644e4abb186fda004466a",            "value": " 792k/792k [00:00&lt;00:00, 23.3MB/s]"          }        },        "e5a556ff241c4621982b2b8d57e295a7": {          "model_module": "@jupyter-widgets/base",          "model_name": "LayoutModel",          "model_module_version": "1.2.0",          "state": {            "_model_module": "@jupyter-widgets/base",            "_model_module_version": "1.2.0",            "_model_name": "LayoutModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/base",            "_view_module_version": "1.2.0",            "_view_name": "LayoutView",            "align_content": null,            "align_items": null,            "align_self": null,            "border": null,            "bottom": null,            "display": null,            "flex": null,            "flex_flow": null,            "grid_area": null,            "grid_auto_columns": null,            "grid_auto_flow": null,            "grid_auto_rows": null,            "grid_column": null,            "grid_gap": null,            "grid_row": null,            "grid_template_areas": null,            "grid_template_columns": null,            "grid_template_rows": null,            "height": null,            "justify_content": null,            "justify_items": null,            "left": null,            "margin": null,            "max_height": null,            "max_width": null,            "min_height": null,            "min_width": null,            "object_fit": null,            "object_position": null,            "order": null,            "overflow": null,            "overflow_x": null,            "overflow_y": null,            "padding": null,            "right": null,            "top": null,            "visibility": null,            "width": null          }        },        "e2ca1a1eea0442819f120b66dcd83053": {          "model_module": "@jupyter-widgets/base",          "model_name": "LayoutModel",          "model_module_version": "1.2.0",          "state": {            "_model_module": "@jupyter-widgets/base",            "_model_module_version": "1.2.0",            "_model_name": "LayoutModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/base",            "_view_module_version": "1.2.0",            "_view_name": "LayoutView",            "align_content": null,            "align_items": null,            "align_self": null,            "border": null,            "bottom": null,            "display": null,            "flex": null,            "flex_flow": null,            "grid_area": null,            "grid_auto_columns": null,            "grid_auto_flow": null,            "grid_auto_rows": null,            "grid_column": null,            "grid_gap": null,            "grid_row": null,            "grid_template_areas": null,            "grid_template_columns": null,            "grid_template_rows": null,            "height": null,            "justify_content": null,            "justify_items": null,            "left": null,            "margin": null,            "max_height": null,            "max_width": null,            "min_height": null,            "min_width": null,            "object_fit": null,            "object_position": null,            "order": null,            "overflow": null,            "overflow_x": null,            "overflow_y": null,            "padding": null,            "right": null,            "top": null,            "visibility": null,            "width": null          }        },        "cc7c897f01cd45338197b4680898a70c": {          "model_module": "@jupyter-widgets/controls",          "model_name": "DescriptionStyleModel",          "model_module_version": "1.5.0",          "state": {            "_model_module": "@jupyter-widgets/controls",            "_model_module_version": "1.5.0",            "_model_name": "DescriptionStyleModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/base",            "_view_module_version": "1.2.0",            "_view_name": "StyleView",            "description_width": ""          }        },        "4c9e9052f056489d9ae3917ea04a1634": {          "model_module": "@jupyter-widgets/base",          "model_name": "LayoutModel",          "model_module_version": "1.2.0",          "state": {            "_model_module": "@jupyter-widgets/base",            "_model_module_version": "1.2.0",            "_model_name": "LayoutModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/base",            "_view_module_version": "1.2.0",            "_view_name": "LayoutView",            "align_content": null,            "align_items": null,            "align_self": null,            "border": null,            "bottom": null,            "display": null,            "flex": null,            "flex_flow": null,            "grid_area": null,            "grid_auto_columns": null,            "grid_auto_flow": null,            "grid_auto_rows": null,            "grid_column": null,            "grid_gap": null,            "grid_row": null,            "grid_template_areas": null,            "grid_template_columns": null,            "grid_template_rows": null,            "height": null,            "justify_content": null,            "justify_items": null,            "left": null,            "margin": null,            "max_height": null,            "max_width": null,            "min_height": null,            "min_width": null,            "object_fit": null,            "object_position": null,            "order": null,            "overflow": null,            "overflow_x": null,            "overflow_y": null,            "padding": null,            "right": null,            "top": null,            "visibility": null,            "width": null          }        },        "549cff618a2a4a0182fe9f4401619207": {          "model_module": "@jupyter-widgets/controls",          "model_name": "ProgressStyleModel",          "model_module_version": "1.5.0",          "state": {            "_model_module": "@jupyter-widgets/controls",            "_model_module_version": "1.5.0",            "_model_name": "ProgressStyleModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/base",            "_view_module_version": "1.2.0",            "_view_name": "StyleView",            "bar_color": null,            "description_width": ""          }        },        "064b297dac5c4067a987bdfdc03988bc": {          "model_module": "@jupyter-widgets/base",          "model_name": "LayoutModel",          "model_module_version": "1.2.0",          "state": {            "_model_module": "@jupyter-widgets/base",            "_model_module_version": "1.2.0",            "_model_name": "LayoutModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/base",            "_view_module_version": "1.2.0",            "_view_name": "LayoutView",            "align_content": null,            "align_items": null,            "align_self": null,            "border": null,            "bottom": null,            "display": null,            "flex": null,            "flex_flow": null,            "grid_area": null,            "grid_auto_columns": null,            "grid_auto_flow": null,            "grid_auto_rows": null,            "grid_column": null,            "grid_gap": null,            "grid_row": null,            "grid_template_areas": null,            "grid_template_columns": null,            "grid_template_rows": null,            "height": null,            "justify_content": null,            "justify_items": null,            "left": null,            "margin": null,            "max_height": null,            "max_width": null,            "min_height": null,            "min_width": null,            "object_fit": null,            "object_position": null,            "order": null,            "overflow": null,            "overflow_x": null,            "overflow_y": null,            "padding": null,            "right": null,            "top": null,            "visibility": null,            "width": null          }        },        "b660155270d644e4abb186fda004466a": {          "model_module": "@jupyter-widgets/controls",          "model_name": "DescriptionStyleModel",          "model_module_version": "1.5.0",          "state": {            "_model_module": "@jupyter-widgets/controls",            "_model_module_version": "1.5.0",            "_model_name": "DescriptionStyleModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/base",            "_view_module_version": "1.2.0",            "_view_name": "StyleView",            "description_width": ""          }        },        "ed16a3bdc8d84e49829d20b75e80f563": {          "model_module": "@jupyter-widgets/controls",          "model_name": "HBoxModel",          "model_module_version": "1.5.0",          "state": {            "_dom_classes": [],            "_model_module": "@jupyter-widgets/controls",            "_model_module_version": "1.5.0",            "_model_name": "HBoxModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/controls",            "_view_module_version": "1.5.0",            "_view_name": "HBoxView",            "box_style": "",            "children": [              "IPY_MODEL_b9fbce71445746abbb697c9e3850a0fc",              "IPY_MODEL_17ddaee8d2374e61a745ad51f43d71fc",              "IPY_MODEL_40543ac91c4a4a4da8b0bda217c4ccb7"            ],            "layout": "IPY_MODEL_9759b9c3826b40a5aa5098578edfa618"          }        },        "b9fbce71445746abbb697c9e3850a0fc": {          "model_module": "@jupyter-widgets/controls",          "model_name": "HTMLModel",          "model_module_version": "1.5.0",          "state": {            "_dom_classes": [],            "_model_module": "@jupyter-widgets/controls",            "_model_module_version": "1.5.0",            "_model_name": "HTMLModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/controls",            "_view_module_version": "1.5.0",            "_view_name": "HTMLView",            "description": "",            "description_tooltip": null,            "layout": "IPY_MODEL_659ed177f79b49e2bc711021e30f055a",            "placeholder": "​",            "style": "IPY_MODEL_38e888ac88a04f1d8f6a99a5732824b9",            "value": "tokenizer.json: "          }        },        "17ddaee8d2374e61a745ad51f43d71fc": {          "model_module": "@jupyter-widgets/controls",          "model_name": "FloatProgressModel",          "model_module_version": "1.5.0",          "state": {            "_dom_classes": [],            "_model_module": "@jupyter-widgets/controls",            "_model_module_version": "1.5.0",            "_model_name": "FloatProgressModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/controls",            "_view_module_version": "1.5.0",            "_view_name": "ProgressView",            "bar_style": "success",            "description": "",            "description_tooltip": null,            "layout": "IPY_MODEL_b99eeb16bc5f4c3cb223ba3d54b8d2db",            "max": 1,            "min": 0,            "orientation": "horizontal",            "style": "IPY_MODEL_4fe9dd10249c4e5a9349d38b48061e14",            "value": 1          }        },        "40543ac91c4a4a4da8b0bda217c4ccb7": {          "model_module": "@jupyter-widgets/controls",          "model_name": "HTMLModel",          "model_module_version": "1.5.0",          "state": {            "_dom_classes": [],            "_model_module": "@jupyter-widgets/controls",            "_model_module_version": "1.5.0",            "_model_name": "HTMLModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/controls",            "_view_module_version": "1.5.0",            "_view_name": "HTMLView",            "description": "",            "description_tooltip": null,            "layout": "IPY_MODEL_636104fca6674fb2942af2acc9e74b97",            "placeholder": "​",            "style": "IPY_MODEL_3a3b8a7a65e649cea8c7103b20f425bf",            "value": " 2.42M/? [00:00&lt;00:00, 29.5MB/s]"          }        },        "9759b9c3826b40a5aa5098578edfa618": {          "model_module": "@jupyter-widgets/base",          "model_name": "LayoutModel",          "model_module_version": "1.2.0",          "state": {            "_model_module": "@jupyter-widgets/base",            "_model_module_version": "1.2.0",            "_model_name": "LayoutModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/base",            "_view_module_version": "1.2.0",            "_view_name": "LayoutView",            "align_content": null,            "align_items": null,            "align_self": null,            "border": null,            "bottom": null,            "display": null,            "flex": null,            "flex_flow": null,            "grid_area": null,            "grid_auto_columns": null,            "grid_auto_flow": null,            "grid_auto_rows": null,            "grid_column": null,            "grid_gap": null,            "grid_row": null,            "grid_template_areas": null,            "grid_template_columns": null,            "grid_template_rows": null,            "height": null,            "justify_content": null,            "justify_items": null,            "left": null,            "margin": null,            "max_height": null,            "max_width": null,            "min_height": null,            "min_width": null,            "object_fit": null,            "object_position": null,            "order": null,            "overflow": null,            "overflow_x": null,            "overflow_y": null,            "padding": null,            "right": null,            "top": null,            "visibility": null,            "width": null          }        },        "659ed177f79b49e2bc711021e30f055a": {          "model_module": "@jupyter-widgets/base",          "model_name": "LayoutModel",          "model_module_version": "1.2.0",          "state": {            "_model_module": "@jupyter-widgets/base",            "_model_module_version": "1.2.0",            "_model_name": "LayoutModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/base",            "_view_module_version": "1.2.0",            "_view_name": "LayoutView",            "align_content": null,            "align_items": null,            "align_self": null,            "border": null,            "bottom": null,            "display": null,            "flex": null,            "flex_flow": null,            "grid_area": null,            "grid_auto_columns": null,            "grid_auto_flow": null,            "grid_auto_rows": null,            "grid_column": null,            "grid_gap": null,            "grid_row": null,            "grid_template_areas": null,            "grid_template_columns": null,            "grid_template_rows": null,            "height": null,            "justify_content": null,            "justify_items": null,            "left": null,            "margin": null,            "max_height": null,            "max_width": null,            "min_height": null,            "min_width": null,            "object_fit": null,            "object_position": null,            "order": null,            "overflow": null,            "overflow_x": null,            "overflow_y": null,            "padding": null,            "right": null,            "top": null,            "visibility": null,            "width": null          }        },        "38e888ac88a04f1d8f6a99a5732824b9": {          "model_module": "@jupyter-widgets/controls",          "model_name": "DescriptionStyleModel",          "model_module_version": "1.5.0",          "state": {            "_model_module": "@jupyter-widgets/controls",            "_model_module_version": "1.5.0",            "_model_name": "DescriptionStyleModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/base",            "_view_module_version": "1.2.0",            "_view_name": "StyleView",            "description_width": ""          }        },        "b99eeb16bc5f4c3cb223ba3d54b8d2db": {          "model_module": "@jupyter-widgets/base",          "model_name": "LayoutModel",          "model_module_version": "1.2.0",          "state": {            "_model_module": "@jupyter-widgets/base",            "_model_module_version": "1.2.0",            "_model_name": "LayoutModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/base",            "_view_module_version": "1.2.0",            "_view_name": "LayoutView",            "align_content": null,            "align_items": null,            "align_self": null,            "border": null,            "bottom": null,            "display": null,            "flex": null,            "flex_flow": null,            "grid_area": null,            "grid_auto_columns": null,            "grid_auto_flow": null,            "grid_auto_rows": null,            "grid_column": null,            "grid_gap": null,            "grid_row": null,            "grid_template_areas": null,            "grid_template_columns": null,            "grid_template_rows": null,            "height": null,            "justify_content": null,            "justify_items": null,            "left": null,            "margin": null,            "max_height": null,            "max_width": null,            "min_height": null,            "min_width": null,            "object_fit": null,            "object_position": null,            "order": null,            "overflow": null,            "overflow_x": null,            "overflow_y": null,            "padding": null,            "right": null,            "top": null,            "visibility": null,            "width": "20px"          }        },        "4fe9dd10249c4e5a9349d38b48061e14": {          "model_module": "@jupyter-widgets/controls",          "model_name": "ProgressStyleModel",          "model_module_version": "1.5.0",          "state": {            "_model_module": "@jupyter-widgets/controls",            "_model_module_version": "1.5.0",            "_model_name": "ProgressStyleModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/base",            "_view_module_version": "1.2.0",            "_view_name": "StyleView",            "bar_color": null,            "description_width": ""          }        },        "636104fca6674fb2942af2acc9e74b97": {          "model_module": "@jupyter-widgets/base",          "model_name": "LayoutModel",          "model_module_version": "1.2.0",          "state": {            "_model_module": "@jupyter-widgets/base",            "_model_module_version": "1.2.0",            "_model_name": "LayoutModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/base",            "_view_module_version": "1.2.0",            "_view_name": "LayoutView",            "align_content": null,            "align_items": null,            "align_self": null,            "border": null,            "bottom": null,            "display": null,            "flex": null,            "flex_flow": null,            "grid_area": null,            "grid_auto_columns": null,            "grid_auto_flow": null,            "grid_auto_rows": null,            "grid_column": null,            "grid_gap": null,            "grid_row": null,            "grid_template_areas": null,            "grid_template_columns": null,            "grid_template_rows": null,            "height": null,            "justify_content": null,            "justify_items": null,            "left": null,            "margin": null,            "max_height": null,            "max_width": null,            "min_height": null,            "min_width": null,            "object_fit": null,            "object_position": null,            "order": null,            "overflow": null,            "overflow_x": null,            "overflow_y": null,            "padding": null,            "right": null,            "top": null,            "visibility": null,            "width": null          }        },        "3a3b8a7a65e649cea8c7103b20f425bf": {          "model_module": "@jupyter-widgets/controls",          "model_name": "DescriptionStyleModel",          "model_module_version": "1.5.0",          "state": {            "_model_module": "@jupyter-widgets/controls",            "_model_module_version": "1.5.0",            "_model_name": "DescriptionStyleModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/base",            "_view_module_version": "1.2.0",            "_view_name": "StyleView",            "description_width": ""          }        },        "7d4a64d4c6ba4ff8ab4e3679be3d5a37": {          "model_module": "@jupyter-widgets/controls",          "model_name": "HBoxModel",          "model_module_version": "1.5.0",          "state": {            "_dom_classes": [],            "_model_module": "@jupyter-widgets/controls",            "_model_module_version": "1.5.0",            "_model_name": "HBoxModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/controls",            "_view_module_version": "1.5.0",            "_view_name": "HBoxView",            "box_style": "",            "children": [              "IPY_MODEL_94651b4dd4f546789824746c975620f2",              "IPY_MODEL_cc526b0d5e314c9a9bb107280fe3bad8",              "IPY_MODEL_85e3422561b242fca101fff36f3ab433"            ],            "layout": "IPY_MODEL_8dc69bdd76e946a3814209baeae194e5"          }        },        "94651b4dd4f546789824746c975620f2": {          "model_module": "@jupyter-widgets/controls",          "model_name": "HTMLModel",          "model_module_version": "1.5.0",          "state": {            "_dom_classes": [],            "_model_module": "@jupyter-widgets/controls",            "_model_module_version": "1.5.0",            "_model_name": "HTMLModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/controls",            "_view_module_version": "1.5.0",            "_view_name": "HTMLView",            "description": "",            "description_tooltip": null,            "layout": "IPY_MODEL_75767740f74240fabd0dd78b666bdd2f",            "placeholder": "​",            "style": "IPY_MODEL_a9d68dab46824807a060eb5037acf196",            "value": "special_tokens_map.json: "          }        },        "cc526b0d5e314c9a9bb107280fe3bad8": {          "model_module": "@jupyter-widgets/controls",          "model_name": "FloatProgressModel",          "model_module_version": "1.5.0",          "state": {            "_dom_classes": [],            "_model_module": "@jupyter-widgets/controls",            "_model_module_version": "1.5.0",            "_model_name": "FloatProgressModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/controls",            "_view_module_version": "1.5.0",            "_view_name": "ProgressView",            "bar_style": "success",            "description": "",            "description_tooltip": null,            "layout": "IPY_MODEL_ef0543f80e144a41bc5c75b7d07de8b4",            "max": 1,            "min": 0,            "orientation": "horizontal",            "style": "IPY_MODEL_6c594f7ae16a4a409579bb8d6d032eeb",            "value": 1          }        },        "85e3422561b242fca101fff36f3ab433": {          "model_module": "@jupyter-widgets/controls",          "model_name": "HTMLModel",          "model_module_version": "1.5.0",          "state": {            "_dom_classes": [],            "_model_module": "@jupyter-widgets/controls",            "_model_module_version": "1.5.0",            "_model_name": "HTMLModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/controls",            "_view_module_version": "1.5.0",            "_view_name": "HTMLView",            "description": "",            "description_tooltip": null,            "layout": "IPY_MODEL_4f5d19e579f14534b64f3b2daa7b65f9",            "placeholder": "​",            "style": "IPY_MODEL_9d82748ecfcd4d99adddc944f45fd3d5",            "value": " 2.20k/? [00:00&lt;00:00, 128kB/s]"          }        },        "8dc69bdd76e946a3814209baeae194e5": {          "model_module": "@jupyter-widgets/base",          "model_name": "LayoutModel",          "model_module_version": "1.2.0",          "state": {            "_model_module": "@jupyter-widgets/base",            "_model_module_version": "1.2.0",            "_model_name": "LayoutModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/base",            "_view_module_version": "1.2.0",            "_view_name": "LayoutView",            "align_content": null,            "align_items": null,            "align_self": null,            "border": null,            "bottom": null,            "display": null,            "flex": null,            "flex_flow": null,            "grid_area": null,            "grid_auto_columns": null,            "grid_auto_flow": null,            "grid_auto_rows": null,            "grid_column": null,            "grid_gap": null,            "grid_row": null,            "grid_template_areas": null,            "grid_template_columns": null,            "grid_template_rows": null,            "height": null,            "justify_content": null,            "justify_items": null,            "left": null,            "margin": null,            "max_height": null,            "max_width": null,            "min_height": null,            "min_width": null,            "object_fit": null,            "object_position": null,            "order": null,            "overflow": null,            "overflow_x": null,            "overflow_y": null,            "padding": null,            "right": null,            "top": null,            "visibility": null,            "width": null          }        },        "75767740f74240fabd0dd78b666bdd2f": {          "model_module": "@jupyter-widgets/base",          "model_name": "LayoutModel",          "model_module_version": "1.2.0",          "state": {            "_model_module": "@jupyter-widgets/base",            "_model_module_version": "1.2.0",            "_model_name": "LayoutModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/base",            "_view_module_version": "1.2.0",            "_view_name": "LayoutView",            "align_content": null,            "align_items": null,            "align_self": null,            "border": null,            "bottom": null,            "display": null,            "flex": null,            "flex_flow": null,            "grid_area": null,            "grid_auto_columns": null,            "grid_auto_flow": null,            "grid_auto_rows": null,            "grid_column": null,            "grid_gap": null,            "grid_row": null,            "grid_template_areas": null,            "grid_template_columns": null,            "grid_template_rows": null,            "height": null,            "justify_content": null,            "justify_items": null,            "left": null,            "margin": null,            "max_height": null,            "max_width": null,            "min_height": null,            "min_width": null,            "object_fit": null,            "object_position": null,            "order": null,            "overflow": null,            "overflow_x": null,            "overflow_y": null,            "padding": null,            "right": null,            "top": null,            "visibility": null,            "width": null          }        },        "a9d68dab46824807a060eb5037acf196": {          "model_module": "@jupyter-widgets/controls",          "model_name": "DescriptionStyleModel",          "model_module_version": "1.5.0",          "state": {            "_model_module": "@jupyter-widgets/controls",            "_model_module_version": "1.5.0",            "_model_name": "DescriptionStyleModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/base",            "_view_module_version": "1.2.0",            "_view_name": "StyleView",            "description_width": ""          }        },        "ef0543f80e144a41bc5c75b7d07de8b4": {          "model_module": "@jupyter-widgets/base",          "model_name": "LayoutModel",          "model_module_version": "1.2.0",          "state": {            "_model_module": "@jupyter-widgets/base",            "_model_module_version": "1.2.0",            "_model_name": "LayoutModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/base",            "_view_module_version": "1.2.0",            "_view_name": "LayoutView",            "align_content": null,            "align_items": null,            "align_self": null,            "border": null,            "bottom": null,            "display": null,            "flex": null,            "flex_flow": null,            "grid_area": null,            "grid_auto_columns": null,            "grid_auto_flow": null,            "grid_auto_rows": null,            "grid_column": null,            "grid_gap": null,            "grid_row": null,            "grid_template_areas": null,            "grid_template_columns": null,            "grid_template_rows": null,            "height": null,            "justify_content": null,            "justify_items": null,            "left": null,            "margin": null,            "max_height": null,            "max_width": null,            "min_height": null,            "min_width": null,            "object_fit": null,            "object_position": null,            "order": null,            "overflow": null,            "overflow_x": null,            "overflow_y": null,            "padding": null,            "right": null,            "top": null,            "visibility": null,            "width": "20px"          }        },        "6c594f7ae16a4a409579bb8d6d032eeb": {          "model_module": "@jupyter-widgets/controls",          "model_name": "ProgressStyleModel",          "model_module_version": "1.5.0",          "state": {            "_model_module": "@jupyter-widgets/controls",            "_model_module_version": "1.5.0",            "_model_name": "ProgressStyleModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/base",            "_view_module_version": "1.2.0",            "_view_name": "StyleView",            "bar_color": null,            "description_width": ""          }        },        "4f5d19e579f14534b64f3b2daa7b65f9": {          "model_module": "@jupyter-widgets/base",          "model_name": "LayoutModel",          "model_module_version": "1.2.0",          "state": {            "_model_module": "@jupyter-widgets/base",            "_model_module_version": "1.2.0",            "_model_name": "LayoutModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/base",            "_view_module_version": "1.2.0",            "_view_name": "LayoutView",            "align_content": null,            "align_items": null,            "align_self": null,            "border": null,            "bottom": null,            "display": null,            "flex": null,            "flex_flow": null,            "grid_area": null,            "grid_auto_columns": null,            "grid_auto_flow": null,            "grid_auto_rows": null,            "grid_column": null,            "grid_gap": null,            "grid_row": null,            "grid_template_areas": null,            "grid_template_columns": null,            "grid_template_rows": null,            "height": null,            "justify_content": null,            "justify_items": null,            "left": null,            "margin": null,            "max_height": null,            "max_width": null,            "min_height": null,            "min_width": null,            "object_fit": null,            "object_position": null,            "order": null,            "overflow": null,            "overflow_x": null,            "overflow_y": null,            "padding": null,            "right": null,            "top": null,            "visibility": null,            "width": null          }        },        "9d82748ecfcd4d99adddc944f45fd3d5": {          "model_module": "@jupyter-widgets/controls",          "model_name": "DescriptionStyleModel",          "model_module_version": "1.5.0",          "state": {            "_model_module": "@jupyter-widgets/controls",            "_model_module_version": "1.5.0",            "_model_name": "DescriptionStyleModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/base",            "_view_module_version": "1.2.0",            "_view_name": "StyleView",            "description_width": ""          }        },        "2a203a71ab9543f08187efc4727ccee2": {          "model_module": "@jupyter-widgets/controls",          "model_name": "HBoxModel",          "model_module_version": "1.5.0",          "state": {            "_dom_classes": [],            "_model_module": "@jupyter-widgets/controls",            "_model_module_version": "1.5.0",            "_model_name": "HBoxModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/controls",            "_view_module_version": "1.5.0",            "_view_name": "HBoxView",            "box_style": "",            "children": [              "IPY_MODEL_e7622d65cf694568bc9aac46e3b0efca",              "IPY_MODEL_3e85c144268e43a08f15c5b3310765cf",              "IPY_MODEL_5247110dfd7943cd96f9d333747f71bd"            ],            "layout": "IPY_MODEL_c145cf17ce8b4f3cad7da30fdb68eadc"          }        },        "e7622d65cf694568bc9aac46e3b0efca": {          "model_module": "@jupyter-widgets/controls",          "model_name": "HTMLModel",          "model_module_version": "1.5.0",          "state": {            "_dom_classes": [],            "_model_module": "@jupyter-widgets/controls",            "_model_module_version": "1.5.0",            "_model_name": "HTMLModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/controls",            "_view_module_version": "1.5.0",            "_view_name": "HTMLView",            "description": "",            "description_tooltip": null,            "layout": "IPY_MODEL_a1bbd4bac87c439992c4b3e156d48a78",            "placeholder": "​",            "style": "IPY_MODEL_ad5203da66eb4509a8bf5e1e1c18dad5",            "value": "config.json: 100%"          }        },        "3e85c144268e43a08f15c5b3310765cf": {          "model_module": "@jupyter-widgets/controls",          "model_name": "FloatProgressModel",          "model_module_version": "1.5.0",          "state": {            "_dom_classes": [],            "_model_module": "@jupyter-widgets/controls",            "_model_module_version": "1.5.0",            "_model_name": "FloatProgressModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/controls",            "_view_module_version": "1.5.0",            "_view_name": "ProgressView",            "bar_style": "success",            "description": "",            "description_tooltip": null,            "layout": "IPY_MODEL_0a54c2ce5dca4b7f9976dbb0278d63b8",            "max": 270,            "min": 0,            "orientation": "horizontal",            "style": "IPY_MODEL_5a88f3855be441fbbba300b3a48fa9bd",            "value": 270          }        },        "5247110dfd7943cd96f9d333747f71bd": {          "model_module": "@jupyter-widgets/controls",          "model_name": "HTMLModel",          "model_module_version": "1.5.0",          "state": {            "_dom_classes": [],            "_model_module": "@jupyter-widgets/controls",            "_model_module_version": "1.5.0",            "_model_name": "HTMLModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/controls",            "_view_module_version": "1.5.0",            "_view_name": "HTMLView",            "description": "",            "description_tooltip": null,            "layout": "IPY_MODEL_ae396983970b4950a018fa50e66ee5e7",            "placeholder": "​",            "style": "IPY_MODEL_ed0a0ec9ea824945bf3dda0b00caee2a",            "value": " 270/270 [00:00&lt;00:00, 16.2kB/s]"          }        },        "c145cf17ce8b4f3cad7da30fdb68eadc": {          "model_module": "@jupyter-widgets/base",          "model_name": "LayoutModel",          "model_module_version": "1.2.0",          "state": {            "_model_module": "@jupyter-widgets/base",            "_model_module_version": "1.2.0",            "_model_name": "LayoutModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/base",            "_view_module_version": "1.2.0",            "_view_name": "LayoutView",            "align_content": null,            "align_items": null,            "align_self": null,            "border": null,            "bottom": null,            "display": null,            "flex": null,            "flex_flow": null,            "grid_area": null,            "grid_auto_columns": null,            "grid_auto_flow": null,            "grid_auto_rows": null,            "grid_column": null,            "grid_gap": null,            "grid_row": null,            "grid_template_areas": null,            "grid_template_columns": null,            "grid_template_rows": null,            "height": null,            "justify_content": null,            "justify_items": null,            "left": null,            "margin": null,            "max_height": null,            "max_width": null,            "min_height": null,            "min_width": null,            "object_fit": null,            "object_position": null,            "order": null,            "overflow": null,            "overflow_x": null,            "overflow_y": null,            "padding": null,            "right": null,            "top": null,            "visibility": null,            "width": null          }        },        "a1bbd4bac87c439992c4b3e156d48a78": {          "model_module": "@jupyter-widgets/base",          "model_name": "LayoutModel",          "model_module_version": "1.2.0",          "state": {            "_model_module": "@jupyter-widgets/base",            "_model_module_version": "1.2.0",            "_model_name": "LayoutModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/base",            "_view_module_version": "1.2.0",            "_view_name": "LayoutView",            "align_content": null,            "align_items": null,            "align_self": null,            "border": null,            "bottom": null,            "display": null,            "flex": null,            "flex_flow": null,            "grid_area": null,            "grid_auto_columns": null,            "grid_auto_flow": null,            "grid_auto_rows": null,            "grid_column": null,            "grid_gap": null,            "grid_row": null,            "grid_template_areas": null,            "grid_template_columns": null,            "grid_template_rows": null,            "height": null,            "justify_content": null,            "justify_items": null,            "left": null,            "margin": null,            "max_height": null,            "max_width": null,            "min_height": null,            "min_width": null,            "object_fit": null,            "object_position": null,            "order": null,            "overflow": null,            "overflow_x": null,            "overflow_y": null,            "padding": null,            "right": null,            "top": null,            "visibility": null,            "width": null          }        },        "ad5203da66eb4509a8bf5e1e1c18dad5": {          "model_module": "@jupyter-widgets/controls",          "model_name": "DescriptionStyleModel",          "model_module_version": "1.5.0",          "state": {            "_model_module": "@jupyter-widgets/controls",            "_model_module_version": "1.5.0",            "_model_name": "DescriptionStyleModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/base",            "_view_module_version": "1.2.0",            "_view_name": "StyleView",            "description_width": ""          }        },        "0a54c2ce5dca4b7f9976dbb0278d63b8": {          "model_module": "@jupyter-widgets/base",          "model_name": "LayoutModel",          "model_module_version": "1.2.0",          "state": {            "_model_module": "@jupyter-widgets/base",            "_model_module_version": "1.2.0",            "_model_name": "LayoutModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/base",            "_view_module_version": "1.2.0",            "_view_name": "LayoutView",            "align_content": null,            "align_items": null,            "align_self": null,            "border": null,            "bottom": null,            "display": null,            "flex": null,            "flex_flow": null,            "grid_area": null,            "grid_auto_columns": null,            "grid_auto_flow": null,            "grid_auto_rows": null,            "grid_column": null,            "grid_gap": null,            "grid_row": null,            "grid_template_areas": null,            "grid_template_columns": null,            "grid_template_rows": null,            "height": null,            "justify_content": null,            "justify_items": null,            "left": null,            "margin": null,            "max_height": null,            "max_width": null,            "min_height": null,            "min_width": null,            "object_fit": null,            "object_position": null,            "order": null,            "overflow": null,            "overflow_x": null,            "overflow_y": null,            "padding": null,            "right": null,            "top": null,            "visibility": null,            "width": null          }        },        "5a88f3855be441fbbba300b3a48fa9bd": {          "model_module": "@jupyter-widgets/controls",          "model_name": "ProgressStyleModel",          "model_module_version": "1.5.0",          "state": {            "_model_module": "@jupyter-widgets/controls",            "_model_module_version": "1.5.0",            "_model_name": "ProgressStyleModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/base",            "_view_module_version": "1.2.0",            "_view_name": "StyleView",            "bar_color": null,            "description_width": ""          }        },        "ae396983970b4950a018fa50e66ee5e7": {          "model_module": "@jupyter-widgets/base",          "model_name": "LayoutModel",          "model_module_version": "1.2.0",          "state": {            "_model_module": "@jupyter-widgets/base",            "_model_module_version": "1.2.0",            "_model_name": "LayoutModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/base",            "_view_module_version": "1.2.0",            "_view_name": "LayoutView",            "align_content": null,            "align_items": null,            "align_self": null,            "border": null,            "bottom": null,            "display": null,            "flex": null,            "flex_flow": null,            "grid_area": null,            "grid_auto_columns": null,            "grid_auto_flow": null,            "grid_auto_rows": null,            "grid_column": null,            "grid_gap": null,            "grid_row": null,            "grid_template_areas": null,            "grid_template_columns": null,            "grid_template_rows": null,            "height": null,            "justify_content": null,            "justify_items": null,            "left": null,            "margin": null,            "max_height": null,            "max_width": null,            "min_height": null,            "min_width": null,            "object_fit": null,            "object_position": null,            "order": null,            "overflow": null,            "overflow_x": null,            "overflow_y": null,            "padding": null,            "right": null,            "top": null,            "visibility": null,            "width": null          }        },        "ed0a0ec9ea824945bf3dda0b00caee2a": {          "model_module": "@jupyter-widgets/controls",          "model_name": "DescriptionStyleModel",          "model_module_version": "1.5.0",          "state": {            "_model_module": "@jupyter-widgets/controls",            "_model_module_version": "1.5.0",            "_model_name": "DescriptionStyleModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/base",            "_view_module_version": "1.2.0",            "_view_name": "StyleView",            "description_width": ""          }        },        "32c2a009a0d444fcb489b63fdd31833d": {          "model_module": "@jupyter-widgets/controls",          "model_name": "HBoxModel",          "model_module_version": "1.5.0",          "state": {            "_dom_classes": [],            "_model_module": "@jupyter-widgets/controls",            "_model_module_version": "1.5.0",            "_model_name": "HBoxModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/controls",            "_view_module_version": "1.5.0",            "_view_name": "HBoxView",            "box_style": "",            "children": [              "IPY_MODEL_96266112a96c4d318f20cad78214aca8",              "IPY_MODEL_2dcd5e47be3940109eedb0ba23cef2e1",              "IPY_MODEL_11d80024a77943d08d149f5812266a7d"            ],            "layout": "IPY_MODEL_2cc0bb4de49844f8a104d0845fa76b54"          }        },        "96266112a96c4d318f20cad78214aca8": {          "model_module": "@jupyter-widgets/controls",          "model_name": "HTMLModel",          "model_module_version": "1.5.0",          "state": {            "_dom_classes": [],            "_model_module": "@jupyter-widgets/controls",            "_model_module_version": "1.5.0",            "_model_name": "HTMLModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/controls",            "_view_module_version": "1.5.0",            "_view_name": "HTMLView",            "description": "",            "description_tooltip": null,            "layout": "IPY_MODEL_dde67741029c487fb3dc5a5c7e099a3c",            "placeholder": "​",            "style": "IPY_MODEL_125b611f34bb4302be41c287983aee7c",            "value": "pytorch_model.bin: 100%"          }        },        "2dcd5e47be3940109eedb0ba23cef2e1": {          "model_module": "@jupyter-widgets/controls",          "model_name": "FloatProgressModel",          "model_module_version": "1.5.0",          "state": {            "_dom_classes": [],            "_model_module": "@jupyter-widgets/controls",            "_model_module_version": "1.5.0",            "_model_name": "FloatProgressModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/controls",            "_view_module_version": "1.5.0",            "_view_name": "ProgressView",            "bar_style": "success",            "description": "",            "description_tooltip": null,            "layout": "IPY_MODEL_fbe250ffa27c445ea253f64cc0afbd83",            "max": 3146603,            "min": 0,            "orientation": "horizontal",            "style": "IPY_MODEL_5a91238b1ef341c487c546469953322d",            "value": 3146603          }        },        "11d80024a77943d08d149f5812266a7d": {          "model_module": "@jupyter-widgets/controls",          "model_name": "HTMLModel",          "model_module_version": "1.5.0",          "state": {            "_dom_classes": [],            "_model_module": "@jupyter-widgets/controls",            "_model_module_version": "1.5.0",            "_model_name": "HTMLModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/controls",            "_view_module_version": "1.5.0",            "_view_name": "HTMLView",            "description": "",            "description_tooltip": null,            "layout": "IPY_MODEL_b4774dbe487342aa9ddbe41fd10fd685",            "placeholder": "​",            "style": "IPY_MODEL_474f6e0a06e341e197fc79f4008d0572",            "value": " 3.15M/3.15M [00:00&lt;00:00, 19.0MB/s]"          }        },        "2cc0bb4de49844f8a104d0845fa76b54": {          "model_module": "@jupyter-widgets/base",          "model_name": "LayoutModel",          "model_module_version": "1.2.0",          "state": {            "_model_module": "@jupyter-widgets/base",            "_model_module_version": "1.2.0",            "_model_name": "LayoutModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/base",            "_view_module_version": "1.2.0",            "_view_name": "LayoutView",            "align_content": null,            "align_items": null,            "align_self": null,            "border": null,            "bottom": null,            "display": null,            "flex": null,            "flex_flow": null,            "grid_area": null,            "grid_auto_columns": null,            "grid_auto_flow": null,            "grid_auto_rows": null,            "grid_column": null,            "grid_gap": null,            "grid_row": null,            "grid_template_areas": null,            "grid_template_columns": null,            "grid_template_rows": null,            "height": null,            "justify_content": null,            "justify_items": null,            "left": null,            "margin": null,            "max_height": null,            "max_width": null,            "min_height": null,            "min_width": null,            "object_fit": null,            "object_position": null,            "order": null,            "overflow": null,            "overflow_x": null,            "overflow_y": null,            "padding": null,            "right": null,            "top": null,            "visibility": null,            "width": null          }        },        "dde67741029c487fb3dc5a5c7e099a3c": {          "model_module": "@jupyter-widgets/base",          "model_name": "LayoutModel",          "model_module_version": "1.2.0",          "state": {            "_model_module": "@jupyter-widgets/base",            "_model_module_version": "1.2.0",            "_model_name": "LayoutModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/base",            "_view_module_version": "1.2.0",            "_view_name": "LayoutView",            "align_content": null,            "align_items": null,            "align_self": null,            "border": null,            "bottom": null,            "display": null,            "flex": null,            "flex_flow": null,            "grid_area": null,            "grid_auto_columns": null,            "grid_auto_flow": null,            "grid_auto_rows": null,            "grid_column": null,            "grid_gap": null,            "grid_row": null,            "grid_template_areas": null,            "grid_template_columns": null,            "grid_template_rows": null,            "height": null,            "justify_content": null,            "justify_items": null,            "left": null,            "margin": null,            "max_height": null,            "max_width": null,            "min_height": null,            "min_width": null,            "object_fit": null,            "object_position": null,            "order": null,            "overflow": null,            "overflow_x": null,            "overflow_y": null,            "padding": null,            "right": null,            "top": null,            "visibility": null,            "width": null          }        },        "125b611f34bb4302be41c287983aee7c": {          "model_module": "@jupyter-widgets/controls",          "model_name": "DescriptionStyleModel",          "model_module_version": "1.5.0",          "state": {            "_model_module": "@jupyter-widgets/controls",            "_model_module_version": "1.5.0",            "_model_name": "DescriptionStyleModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/base",            "_view_module_version": "1.2.0",            "_view_name": "StyleView",            "description_width": ""          }        },        "fbe250ffa27c445ea253f64cc0afbd83": {          "model_module": "@jupyter-widgets/base",          "model_name": "LayoutModel",          "model_module_version": "1.2.0",          "state": {            "_model_module": "@jupyter-widgets/base",            "_model_module_version": "1.2.0",            "_model_name": "LayoutModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/base",            "_view_module_version": "1.2.0",            "_view_name": "LayoutView",            "align_content": null,            "align_items": null,            "align_self": null,            "border": null,            "bottom": null,            "display": null,            "flex": null,            "flex_flow": null,            "grid_area": null,            "grid_auto_columns": null,            "grid_auto_flow": null,            "grid_auto_rows": null,            "grid_column": null,            "grid_gap": null,            "grid_row": null,            "grid_template_areas": null,            "grid_template_columns": null,            "grid_template_rows": null,            "height": null,            "justify_content": null,            "justify_items": null,            "left": null,            "margin": null,            "max_height": null,            "max_width": null,            "min_height": null,            "min_width": null,            "object_fit": null,            "object_position": null,            "order": null,            "overflow": null,            "overflow_x": null,            "overflow_y": null,            "padding": null,            "right": null,            "top": null,            "visibility": null,            "width": null          }        },        "5a91238b1ef341c487c546469953322d": {          "model_module": "@jupyter-widgets/controls",          "model_name": "ProgressStyleModel",          "model_module_version": "1.5.0",          "state": {            "_model_module": "@jupyter-widgets/controls",            "_model_module_version": "1.5.0",            "_model_name": "ProgressStyleModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/base",            "_view_module_version": "1.2.0",            "_view_name": "StyleView",            "bar_color": null,            "description_width": ""          }        },        "b4774dbe487342aa9ddbe41fd10fd685": {          "model_module": "@jupyter-widgets/base",          "model_name": "LayoutModel",          "model_module_version": "1.2.0",          "state": {            "_model_module": "@jupyter-widgets/base",            "_model_module_version": "1.2.0",            "_model_name": "LayoutModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/base",            "_view_module_version": "1.2.0",            "_view_name": "LayoutView",            "align_content": null,            "align_items": null,            "align_self": null,            "border": null,            "bottom": null,            "display": null,            "flex": null,            "flex_flow": null,            "grid_area": null,            "grid_auto_columns": null,            "grid_auto_flow": null,            "grid_auto_rows": null,            "grid_column": null,            "grid_gap": null,            "grid_row": null,            "grid_template_areas": null,            "grid_template_columns": null,            "grid_template_rows": null,            "height": null,            "justify_content": null,            "justify_items": null,            "left": null,            "margin": null,            "max_height": null,            "max_width": null,            "min_height": null,            "min_width": null,            "object_fit": null,            "object_position": null,            "order": null,            "overflow": null,            "overflow_x": null,            "overflow_y": null,            "padding": null,            "right": null,            "top": null,            "visibility": null,            "width": null          }        },        "474f6e0a06e341e197fc79f4008d0572": {          "model_module": "@jupyter-widgets/controls",          "model_name": "DescriptionStyleModel",          "model_module_version": "1.5.0",          "state": {            "_model_module": "@jupyter-widgets/controls",            "_model_module_version": "1.5.0",            "_model_name": "DescriptionStyleModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/base",            "_view_module_version": "1.2.0",            "_view_name": "StyleView",            "description_width": ""          }        },        "508f236bf682419cbafbb5ce63e0f515": {          "model_module": "@jupyter-widgets/controls",          "model_name": "HBoxModel",          "model_module_version": "1.5.0",          "state": {            "_dom_classes": [],            "_model_module": "@jupyter-widgets/controls",            "_model_module_version": "1.5.0",            "_model_name": "HBoxModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/controls",            "_view_module_version": "1.5.0",            "_view_name": "HBoxView",            "box_style": "",            "children": [              "IPY_MODEL_d9cca8d159274e4cae1f94400813dff6",              "IPY_MODEL_3624b56fb27248288826896c61934d58",              "IPY_MODEL_4c38048133024510be7cd143252c6ceb"            ],            "layout": "IPY_MODEL_0e51d089c63e4e1aaf2e874c271a2b98"          }        },        "d9cca8d159274e4cae1f94400813dff6": {          "model_module": "@jupyter-widgets/controls",          "model_name": "HTMLModel",          "model_module_version": "1.5.0",          "state": {            "_dom_classes": [],            "_model_module": "@jupyter-widgets/controls",            "_model_module_version": "1.5.0",            "_model_name": "HTMLModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/controls",            "_view_module_version": "1.5.0",            "_view_name": "HTMLView",            "description": "",            "description_tooltip": null,            "layout": "IPY_MODEL_8af033d4969e4f348ed8532e5a0e2817",            "placeholder": "​",            "style": "IPY_MODEL_c25b81d0d41245b28b7ada2219079111",            "value": "config.json: 100%"          }        },        "3624b56fb27248288826896c61934d58": {          "model_module": "@jupyter-widgets/controls",          "model_name": "FloatProgressModel",          "model_module_version": "1.5.0",          "state": {            "_dom_classes": [],            "_model_module": "@jupyter-widgets/controls",            "_model_module_version": "1.5.0",            "_model_name": "FloatProgressModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/controls",            "_view_module_version": "1.5.0",            "_view_name": "ProgressView",            "bar_style": "success",            "description": "",            "description_tooltip": null,            "layout": "IPY_MODEL_11dc04dab55044058f6a4eecc7c70d82",            "max": 116,            "min": 0,            "orientation": "horizontal",            "style": "IPY_MODEL_030e9279a594403aaf19b48f5470e116",            "value": 116          }        },        "4c38048133024510be7cd143252c6ceb": {          "model_module": "@jupyter-widgets/controls",          "model_name": "HTMLModel",          "model_module_version": "1.5.0",          "state": {            "_dom_classes": [],            "_model_module": "@jupyter-widgets/controls",            "_model_module_version": "1.5.0",            "_model_name": "HTMLModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/controls",            "_view_module_version": "1.5.0",            "_view_name": "HTMLView",            "description": "",            "description_tooltip": null,            "layout": "IPY_MODEL_5bf8cedb2a90426895d601e7a3bd1bb1",            "placeholder": "​",            "style": "IPY_MODEL_bd08a37cff63452dbe6c58965cbbbcd2",            "value": " 116/116 [00:00&lt;00:00, 7.30kB/s]"          }        },        "0e51d089c63e4e1aaf2e874c271a2b98": {          "model_module": "@jupyter-widgets/base",          "model_name": "LayoutModel",          "model_module_version": "1.2.0",          "state": {            "_model_module": "@jupyter-widgets/base",            "_model_module_version": "1.2.0",            "_model_name": "LayoutModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/base",            "_view_module_version": "1.2.0",            "_view_name": "LayoutView",            "align_content": null,            "align_items": null,            "align_self": null,            "border": null,            "bottom": null,            "display": null,            "flex": null,            "flex_flow": null,            "grid_area": null,            "grid_auto_columns": null,            "grid_auto_flow": null,            "grid_auto_rows": null,            "grid_column": null,            "grid_gap": null,            "grid_row": null,            "grid_template_areas": null,            "grid_template_columns": null,            "grid_template_rows": null,            "height": null,            "justify_content": null,            "justify_items": null,            "left": null,            "margin": null,            "max_height": null,            "max_width": null,            "min_height": null,            "min_width": null,            "object_fit": null,            "object_position": null,            "order": null,            "overflow": null,            "overflow_x": null,            "overflow_y": null,            "padding": null,            "right": null,            "top": null,            "visibility": null,            "width": null          }        },        "8af033d4969e4f348ed8532e5a0e2817": {          "model_module": "@jupyter-widgets/base",          "model_name": "LayoutModel",          "model_module_version": "1.2.0",          "state": {            "_model_module": "@jupyter-widgets/base",            "_model_module_version": "1.2.0",            "_model_name": "LayoutModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/base",            "_view_module_version": "1.2.0",            "_view_name": "LayoutView",            "align_content": null,            "align_items": null,            "align_self": null,            "border": null,            "bottom": null,            "display": null,            "flex": null,            "flex_flow": null,            "grid_area": null,            "grid_auto_columns": null,            "grid_auto_flow": null,            "grid_auto_rows": null,            "grid_column": null,            "grid_gap": null,            "grid_row": null,            "grid_template_areas": null,            "grid_template_columns": null,            "grid_template_rows": null,            "height": null,            "justify_content": null,            "justify_items": null,            "left": null,            "margin": null,            "max_height": null,            "max_width": null,            "min_height": null,            "min_width": null,            "object_fit": null,            "object_position": null,            "order": null,            "overflow": null,            "overflow_x": null,            "overflow_y": null,            "padding": null,            "right": null,            "top": null,            "visibility": null,            "width": null          }        },        "c25b81d0d41245b28b7ada2219079111": {          "model_module": "@jupyter-widgets/controls",          "model_name": "DescriptionStyleModel",          "model_module_version": "1.5.0",          "state": {            "_model_module": "@jupyter-widgets/controls",            "_model_module_version": "1.5.0",            "_model_name": "DescriptionStyleModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/base",            "_view_module_version": "1.2.0",            "_view_name": "StyleView",            "description_width": ""          }        },        "11dc04dab55044058f6a4eecc7c70d82": {          "model_module": "@jupyter-widgets/base",          "model_name": "LayoutModel",          "model_module_version": "1.2.0",          "state": {            "_model_module": "@jupyter-widgets/base",            "_model_module_version": "1.2.0",            "_model_name": "LayoutModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/base",            "_view_module_version": "1.2.0",            "_view_name": "LayoutView",            "align_content": null,            "align_items": null,            "align_self": null,            "border": null,            "bottom": null,            "display": null,            "flex": null,            "flex_flow": null,            "grid_area": null,            "grid_auto_columns": null,            "grid_auto_flow": null,            "grid_auto_rows": null,            "grid_column": null,            "grid_gap": null,            "grid_row": null,            "grid_template_areas": null,            "grid_template_columns": null,            "grid_template_rows": null,            "height": null,            "justify_content": null,            "justify_items": null,            "left": null,            "margin": null,            "max_height": null,            "max_width": null,            "min_height": null,            "min_width": null,            "object_fit": null,            "object_position": null,            "order": null,            "overflow": null,            "overflow_x": null,            "overflow_y": null,            "padding": null,            "right": null,            "top": null,            "visibility": null,            "width": null          }        },        "030e9279a594403aaf19b48f5470e116": {          "model_module": "@jupyter-widgets/controls",          "model_name": "ProgressStyleModel",          "model_module_version": "1.5.0",          "state": {            "_model_module": "@jupyter-widgets/controls",            "_model_module_version": "1.5.0",            "_model_name": "ProgressStyleModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/base",            "_view_module_version": "1.2.0",            "_view_name": "StyleView",            "bar_color": null,            "description_width": ""          }        },        "5bf8cedb2a90426895d601e7a3bd1bb1": {          "model_module": "@jupyter-widgets/base",          "model_name": "LayoutModel",          "model_module_version": "1.2.0",          "state": {            "_model_module": "@jupyter-widgets/base",            "_model_module_version": "1.2.0",            "_model_name": "LayoutModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/base",            "_view_module_version": "1.2.0",            "_view_name": "LayoutView",            "align_content": null,            "align_items": null,            "align_self": null,            "border": null,            "bottom": null,            "display": null,            "flex": null,            "flex_flow": null,            "grid_area": null,            "grid_auto_columns": null,            "grid_auto_flow": null,            "grid_auto_rows": null,            "grid_column": null,            "grid_gap": null,            "grid_row": null,            "grid_template_areas": null,            "grid_template_columns": null,            "grid_template_rows": null,            "height": null,            "justify_content": null,            "justify_items": null,            "left": null,            "margin": null,            "max_height": null,            "max_width": null,            "min_height": null,            "min_width": null,            "object_fit": null,            "object_position": null,            "order": null,            "overflow": null,            "overflow_x": null,            "overflow_y": null,            "padding": null,            "right": null,            "top": null,            "visibility": null,            "width": null          }        },        "bd08a37cff63452dbe6c58965cbbbcd2": {          "model_module": "@jupyter-widgets/controls",          "model_name": "DescriptionStyleModel",          "model_module_version": "1.5.0",          "state": {            "_model_module": "@jupyter-widgets/controls",            "_model_module_version": "1.5.0",            "_model_name": "DescriptionStyleModel",            "_view_count": null,            "_view_module": "@jupyter-widgets/base",            "_view_module_version": "1.2.0",            "_view_name": "StyleView",            "description_width": ""          }        }      }    }  },  "cells": [    {      "cell_type": "code",      "execution_count": null,      "metadata": {        "colab": {          "base_uri": "https://localhost:8080/"        },        "id": "12bKYCBcsL4M",        "outputId": "6f964f20-e86c-426d-f07d-7e8d13463c80"      },      "outputs": [        {          "output_type": "stream",          "name": "stdout",          "text": [            "Collecting langchain-openai\n",            "  Downloading langchain_openai-0.3.27-py3-none-any.whl.metadata (2.3 kB)\n",            "Collecting langchain-community\n",            "  Downloading langchain_community-0.3.26-py3-none-any.whl.metadata (2.9 kB)\n",            "Requirement already satisfied: huggingface_hub in /usr/local/lib/python3.11/dist-packages (0.33.0)\n",            "Collecting PyPDF2\n",            "  Downloading pypdf2-3.0.1-py3-none-any.whl.metadata (6.8 kB)\n",            "Collecting langchain-huggingface\n",            "  Downloading langchain_huggingface-0.3.0-py3-none-any.whl.metadata (996 bytes)\n",            "Collecting faiss-cpu\n",            "  Downloading faiss_cpu-1.11.0-cp311-cp311-manylinux_2_28_x86_64.whl.metadata (4.8 kB)\n",            "Collecting langchain-groq\n",            "  Downloading langchain_groq-0.3.4-py3-none-any.whl.metadata (2.6 kB)\n",            "Requirement already satisfied: langchain-core<1.0.0,>=0.3.66 in /usr/local/lib/python3.11/dist-packages (from langchain-openai) (0.3.66)\n",            "Requirement already satisfied: openai<2.0.0,>=1.86.0 in /usr/local/lib/python3.11/dist-packages (from langchain-openai) (1.91.0)\n",            "Requirement already satisfied: tiktoken<1,>=0.7 in /usr/local/lib/python3.11/dist-packages (from langchain-openai) (0.9.0)\n",            "Requirement already satisfied: langchain<1.0.0,>=0.3.26 in /usr/local/lib/python3.11/dist-packages (from langchain-community) (0.3.26)\n",            "Requirement already satisfied: SQLAlchemy<3,>=1.4 in /usr/local/lib/python3.11/dist-packages (from langchain-community) (2.0.41)\n",            "Requirement already satisfied: requests<3,>=2 in /usr/local/lib/python3.11/dist-packages (from langchain-community) (2.32.3)\n",            "Requirement already satisfied: PyYAML>=5.3 in /usr/local/lib/python3.11/dist-packages (from langchain-community) (6.0.2)\n",            "Requirement already satisfied: aiohttp<4.0.0,>=3.8.3 in /usr/local/lib/python3.11/dist-packages (from langchain-community) (3.11.15)\n",            "Requirement already satisfied: tenacity!=8.4.0,<10,>=8.1.0 in /usr/local/lib/python3.11/dist-packages (from langchain-community) (8.5.0)\n",            "Collecting dataclasses-json<0.7,>=0.5.7 (from langchain-community)\n",            "  Downloading dataclasses_json-0.6.7-py3-none-any.whl.metadata (25 kB)\n",            "Collecting pydantic-settings<3.0.0,>=2.4.0 (from langchain-community)\n",            "  Downloading pydantic_settings-2.10.1-py3-none-any.whl.metadata (3.4 kB)\n",            "Requirement already satisfied: langsmith>=0.1.125 in /usr/local/lib/python3.11/dist-packages (from langchain-community) (0.4.1)\n",            "Collecting httpx-sse<1.0.0,>=0.4.0 (from langchain-community)\n",            "  Downloading httpx_sse-0.4.1-py3-none-any.whl.metadata (9.4 kB)\n",            "Requirement already satisfied: numpy>=1.26.2 in /usr/local/lib/python3.11/dist-packages (from langchain-community) (2.0.2)\n",            "Requirement already satisfied: filelock in /usr/local/lib/python3.11/dist-packages (from huggingface_hub) (3.18.0)\n",            "Requirement already satisfied: fsspec>=2023.5.0 in /usr/local/lib/python3.11/dist-packages (from huggingface_hub) (2025.3.2)\n",            "Requirement already satisfied: packaging>=20.9 in /usr/local/lib/python3.11/dist-packages (from huggingface_hub) (24.2)\n",            "Requirement already satisfied: tqdm>=4.42.1 in /usr/local/lib/python3.11/dist-packages (from huggingface_hub) (4.67.1)\n",            "Requirement already satisfied: typing-extensions>=3.7.4.3 in /usr/local/lib/python3.11/dist-packages (from huggingface_hub) (4.14.0)\n",            "Requirement already satisfied: hf-xet<2.0.0,>=1.1.2 in /usr/local/lib/python3.11/dist-packages (from huggingface_hub) (1.1.5)\n",            "Requirement already satisfied: tokenizers>=0.19.1 in /usr/local/lib/python3.11/dist-packages (from langchain-huggingface) (0.21.2)\n",            "Collecting groq<1,>=0.28.0 (from langchain-groq)\n",            "  Downloading groq-0.29.0-py3-none-any.whl.metadata (16 kB)\n",            "Requirement already satisfied: aiohappyeyeballs>=2.3.0 in /usr/local/lib/python3.11/dist-packages (from aiohttp<4.0.0,>=3.8.3->langchain-community) (2.6.1)\n",            "Requirement already satisfied: aiosignal>=1.1.2 in /usr/local/lib/python3.11/dist-packages (from aiohttp<4.0.0,>=3.8.3->langchain-community) (1.3.2)\n",            "Requirement already satisfied: attrs>=17.3.0 in /usr/local/lib/python3.11/dist-packages (from aiohttp<4.0.0,>=3.8.3->langchain-community) (25.3.0)\n",            "Requirement already satisfied: frozenlist>=1.1.1 in /usr/local/lib/python3.11/dist-packages (from aiohttp<4.0.0,>=3.8.3->langchain-community) (1.7.0)\n",            "Requirement already satisfied: multidict<7.0,>=4.5 in /usr/local/lib/python3.11/dist-packages (from aiohttp<4.0.0,>=3.8.3->langchain-community) (6.4.4)\n",            "Requirement already satisfied: propcache>=0.2.0 in /usr/local/lib/python3.11/dist-packages (from aiohttp<4.0.0,>=3.8.3->langchain-community) (0.3.2)\n",            "Requirement already satisfied: yarl<2.0,>=1.17.0 in /usr/local/lib/python3.11/dist-packages (from aiohttp<4.0.0,>=3.8.3->langchain-community) (1.20.1)\n",            "Collecting marshmallow<4.0.0,>=3.18.0 (from dataclasses-json<0.7,>=0.5.7->langchain-community)\n",            "  Downloading marshmallow-3.26.1-py3-none-any.whl.metadata (7.3 kB)\n",            "Collecting typing-inspect<1,>=0.4.0 (from dataclasses-json<0.7,>=0.5.7->langchain-community)\n",            "  Downloading typing_inspect-0.9.0-py3-none-any.whl.metadata (1.5 kB)\n",            "Requirement already satisfied: anyio<5,>=3.5.0 in /usr/local/lib/python3.11/dist-packages (from groq<1,>=0.28.0->langchain-groq) (4.9.0)\n",            "Requirement already satisfied: distro<2,>=1.7.0 in /usr/local/lib/python3.11/dist-packages (from groq<1,>=0.28.0->langchain-groq) (1.9.0)\n",            "Requirement already satisfied: httpx<1,>=0.23.0 in /usr/local/lib/python3.11/dist-packages (from groq<1,>=0.28.0->langchain-groq) (0.28.1)\n",            "Requirement already satisfied: pydantic<3,>=1.9.0 in /usr/local/lib/python3.11/dist-packages (from groq<1,>=0.28.0->langchain-groq) (2.11.7)\n",            "Requirement already satisfied: sniffio in /usr/local/lib/python3.11/dist-packages (from groq<1,>=0.28.0->langchain-groq) (1.3.1)\n",            "Requirement already satisfied: langchain-text-splitters<1.0.0,>=0.3.8 in /usr/local/lib/python3.11/dist-packages (from langchain<1.0.0,>=0.3.26->langchain-community) (0.3.8)\n",            "Requirement already satisfied: jsonpatch<2.0,>=1.33 in /usr/local/lib/python3.11/dist-packages (from langchain-core<1.0.0,>=0.3.66->langchain-openai) (1.33)\n",            "Requirement already satisfied: orjson<4.0.0,>=3.9.14 in /usr/local/lib/python3.11/dist-packages (from langsmith>=0.1.125->langchain-community) (3.10.18)\n",            "Requirement already satisfied: requests-toolbelt<2.0.0,>=1.0.0 in /usr/local/lib/python3.11/dist-packages (from langsmith>=0.1.125->langchain-community) (1.0.0)\n",            "Requirement already satisfied: zstandard<0.24.0,>=0.23.0 in /usr/local/lib/python3.11/dist-packages (from langsmith>=0.1.125->langchain-community) (0.23.0)\n",            "Requirement already satisfied: jiter<1,>=0.4.0 in /usr/local/lib/python3.11/dist-packages (from openai<2.0.0,>=1.86.0->langchain-openai) (0.10.0)\n",            "Requirement already satisfied: python-dotenv>=0.21.0 in /usr/local/lib/python3.11/dist-packages (from pydantic-settings<3.0.0,>=2.4.0->langchain-community) (1.1.1)\n",            "Requirement already satisfied: typing-inspection>=0.4.0 in /usr/local/lib/python3.11/dist-packages (from pydantic-settings<3.0.0,>=2.4.0->langchain-community) (0.4.1)\n",            "Requirement already satisfied: charset-normalizer<4,>=2 in /usr/local/lib/python3.11/dist-packages (from requests<3,>=2->langchain-community) (3.4.2)\n",            "Requirement already satisfied: idna<4,>=2.5 in /usr/local/lib/python3.11/dist-packages (from requests<3,>=2->langchain-community) (3.10)\n",            "Requirement already satisfied: urllib3<3,>=1.21.1 in /usr/local/lib/python3.11/dist-packages (from requests<3,>=2->langchain-community) (2.4.0)\n",            "Requirement already satisfied: certifi>=2017.4.17 in /usr/local/lib/python3.11/dist-packages (from requests<3,>=2->langchain-community) (2025.6.15)\n",            "Requirement already satisfied: greenlet>=1 in /usr/local/lib/python3.11/dist-packages (from SQLAlchemy<3,>=1.4->langchain-community) (3.2.3)\n",            "Requirement already satisfied: regex>=2022.1.18 in /usr/local/lib/python3.11/dist-packages (from tiktoken<1,>=0.7->langchain-openai) (2024.11.6)\n",            "Requirement already satisfied: httpcore==1.* in /usr/local/lib/python3.11/dist-packages (from httpx<1,>=0.23.0->groq<1,>=0.28.0->langchain-groq) (1.0.9)\n",            "Requirement already satisfied: h11>=0.16 in /usr/local/lib/python3.11/dist-packages (from httpcore==1.*->httpx<1,>=0.23.0->groq<1,>=0.28.0->langchain-groq) (0.16.0)\n",            "Requirement already satisfied: jsonpointer>=1.9 in /usr/local/lib/python3.11/dist-packages (from jsonpatch<2.0,>=1.33->langchain-core<1.0.0,>=0.3.66->langchain-openai) (3.0.0)\n",            "Requirement already satisfied: annotated-types>=0.6.0 in /usr/local/lib/python3.11/dist-packages (from pydantic<3,>=1.9.0->groq<1,>=0.28.0->langchain-groq) (0.7.0)\n",            "Requirement already satisfied: pydantic-core==2.33.2 in /usr/local/lib/python3.11/dist-packages (from pydantic<3,>=1.9.0->groq<1,>=0.28.0->langchain-groq) (2.33.2)\n",            "Collecting mypy-extensions>=0.3.0 (from typing-inspect<1,>=0.4.0->dataclasses-json<0.7,>=0.5.7->langchain-community)\n",            "  Downloading mypy_extensions-1.1.0-py3-none-any.whl.metadata (1.1 kB)\n",            "Downloading langchain_openai-0.3.27-py3-none-any.whl (70 kB)\n",            "\u001b[2K   \u001b[90m━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\u001b[0m \u001b[32m70.4/70.4 kB\u001b[0m \u001b[31m4.3 MB/s\u001b[0m eta \u001b[36m0:00:00\u001b[0m\n",            "\u001b[?25hDownloading langchain_community-0.3.26-py3-none-any.whl (2.5 MB)\n",            "\u001b[2K   \u001b[90m━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\u001b[0m \u001b[32m2.5/2.5 MB\u001b[0m \u001b[31m53.5 MB/s\u001b[0m eta \u001b[36m0:00:00\u001b[0m\n",            "\u001b[?25hDownloading pypdf2-3.0.1-py3-none-any.whl (232 kB)\n",            "\u001b[2K   \u001b[90m━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\u001b[0m \u001b[32m232.6/232.6 kB\u001b[0m \u001b[31m22.0 MB/s\u001b[0m eta \u001b[36m0:00:00\u001b[0m\n",            "\u001b[?25hDownloading langchain_huggingface-0.3.0-py3-none-any.whl (27 kB)\n",            "Downloading faiss_cpu-1.11.0-cp311-cp311-manylinux_2_28_x86_64.whl (31.3 MB)\n",            "\u001b[2K   \u001b[90m━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\u001b[0m \u001b[32m31.3/31.3 MB\u001b[0m \u001b[31m67.5 MB/s\u001b[0m eta \u001b[36m0:00:00\u001b[0m\n",            "\u001b[?25hDownloading langchain_groq-0.3.4-py3-none-any.whl (15 kB)\n",            "Downloading dataclasses_json-0.6.7-py3-none-any.whl (28 kB)\n",            "Downloading groq-0.29.0-py3-none-any.whl (130 kB)\n",            "\u001b[2K   \u001b[90m━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\u001b[0m \u001b[32m130.8/130.8 kB\u001b[0m \u001b[31m15.7 MB/s\u001b[0m eta \u001b[36m0:00:00\u001b[0m\n",            "\u001b[?25hDownloading httpx_sse-0.4.1-py3-none-any.whl (8.1 kB)\n",            "Downloading pydantic_settings-2.10.1-py3-none-any.whl (45 kB)\n",            "\u001b[2K   \u001b[90m━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\u001b[0m \u001b[32m45.2/45.2 kB\u001b[0m \u001b[31m4.7 MB/s\u001b[0m eta \u001b[36m0:00:00\u001b[0m\n",            "\u001b[?25hDownloading marshmallow-3.26.1-py3-none-any.whl (50 kB)\n",            "\u001b[2K   \u001b[90m━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\u001b[0m \u001b[32m50.9/50.9 kB\u001b[0m \u001b[31m5.6 MB/s\u001b[0m eta \u001b[36m0:00:00\u001b[0m\n",            "\u001b[?25hDownloading typing_inspect-0.9.0-py3-none-any.whl (8.8 kB)\n",            "Downloading mypy_extensions-1.1.0-py3-none-any.whl (5.0 kB)\n",            "Installing collected packages: PyPDF2, mypy-extensions, marshmallow, httpx-sse, faiss-cpu, typing-inspect, pydantic-settings, groq, dataclasses-json, langchain-openai, langchain-huggingface, langchain-groq, langchain-community\n",            "Successfully installed PyPDF2-3.0.1 dataclasses-json-0.6.7 faiss-cpu-1.11.0 groq-0.29.0 httpx-sse-0.4.1 langchain-community-0.3.26 langchain-groq-0.3.4 langchain-huggingface-0.3.0 langchain-openai-0.3.27 marshmallow-3.26.1 mypy-extensions-1.1.0 pydantic-settings-2.10.1 typing-inspect-0.9.0\n"          ]        }      ],      "source": [        "!pip install langchain-openai langchain-community huggingface_hub PyPDF2 langchain-huggingface faiss-cpu langchain-groq"      ]    },    {      "cell_type": "code",      "source": [        "!pip install dotenv"      ],      "metadata": {        "colab": {          "base_uri": "https://localhost:8080/"        },        "id": "3mMGJkfgtsjK",        "outputId": "19dcca81-1c1d-4fcf-ac4a-6ff684d18376"      },      "execution_count": null,      "outputs": [        {          "output_type": "stream",          "name": "stdout",          "text": [            "Collecting dotenv\n",            "  Downloading dotenv-0.9.9-py2.py3-none-any.whl.metadata (279 bytes)\n",            "Collecting python-dotenv (from dotenv)\n",            "  Downloading python_dotenv-1.1.1-py3-none-any.whl.metadata (24 kB)\n",            "Downloading dotenv-0.9.9-py2.py3-none-any.whl (1.9 kB)\n",            "Downloading python_dotenv-1.1.1-py3-none-any.whl (20 kB)\n",            "Installing collected packages: python-dotenv, dotenv\n",            "Successfully installed dotenv-0.9.9 python-dotenv-1.1.1\n"          ]        }      ]    },    {      "cell_type": "code",      "source": [        "from dotenv import load_dotenv\n",        "from PyPDF2 import PdfReader\n",        "from langchain.text_splitter import CharacterTextSplitter\n",        "from langchain_huggingface import HuggingFaceEmbeddings\n",        "from langchain_openai import ChatOpenAI\n",        "from langchain.vectorstores import FAISS\n",        "from langchain.memory import ConversationBufferMemory\n",        "from langchain.chains import ConversationalRetrievalChain\n",        "from huggingface_hub import login\n",        "from langchain_groq import ChatGroq\n",        "import os"      ],      "metadata": {        "id": "zD0I-h1FsP_y"      },      "execution_count": null,      "outputs": []    },    {      "cell_type": "code",      "source": [        "load_dotenv()\n",        "login(os.getenv(\"HUGGINGFACEHUB_API_TOKEN\"))"      ],      "metadata": {        "colab": {          "base_uri": "https://localhost:8080/",          "height": 17,          "referenced_widgets": [            "f5108df1194646538efe27b2c502ec62",            "0e4cdff0a2cd40e6bef99ae8153f1bfb",            "45ede06fa69e4d8a8dde12c563c742e3",            "8420565a9f0b459cb9a4763379b13851",            "83cf4129d1434849b8763fda880b0e2c",            "e99236bcdb5d477f88be5bfd70cc32f9",            "8418570205dc416083b062bf580345ae",            "f49da97483f844a49610208e9251866a",            "e22233b61029425d9c01f695672f5569",            "ba3b87ac7d7b4960a3fe138c05f99d46",            "2eda0349ffe84ccc8b89a0497bc124fe",            "c67e328b579847379409cd470488184e",            "33b84329f06646859821d7494989758c",            "3f272cbed3f949af9a6f961a83131ed1",            "36e19d253d6146a9aae40742044c5fa5",            "c210b08d990c4f4e94f23f3a152b8b83",            "ed6f78b90310486781bcaa82e9349098",            "8d1cda37e82b4d20bf885aadcb6b15d4",            "9d33802ae3274dc8aa9d796a379c03c9",            "eafdfe329aa747e7833841e11b141183"          ]        },        "id": "yV-F8DzxsQ7c",        "outputId": "2edf4fd5-b3da-4637-8199-00ba2d1e53fa"      },      "execution_count": null,      "outputs": [        {          "output_type": "display_data",          "data": {            "text/plain": [              "VBox(children=(HTML(value='<center> <img\\nsrc=https://huggingface.co/front/assets/huggingface_logo-noborder.sv…"            ],            "application/vnd.jupyter.widget-view+json": {              "version_major": 2,              "version_minor": 0,              "model_id": "f5108df1194646538efe27b2c502ec62"            }          },          "metadata": {}        }      ]    },    {      "cell_type": "code",      "source": [        "from google.colab import files\n",        "uploaded = files.upload()"      ],      "metadata": {        "colab": {          "base_uri": "https://localhost:8080/",          "height": 73        },        "id": "NWkaJ8xLsT1x",        "outputId": "5666bc71-3af7-40cb-c02d-31e5ce09a99f"      },      "execution_count": null,      "outputs": [        {          "output_type": "display_data",          "data": {            "text/plain": [              "<IPython.core.display.HTML object>"            ],            "text/html": [              "\n",              "     <input type=\"file\" id=\"files-d3cc6ab7-c958-4ba6-ab68-a95183dca7ed\" name=\"files[]\" multiple disabled\n",              "        style=\"border:none\" />\n",              "     <output id=\"result-d3cc6ab7-c958-4ba6-ab68-a95183dca7ed\">\n",              "      Upload widget is only available when the cell has been executed in the\n",              "      current browser session. Please rerun this cell to enable.\n",              "      </output>\n",              "      <script>// Copyright 2017 Google LLC\n",              "//\n",              "// Licensed under the Apache License, Version 2.0 (the \"License\");\n",              "// you may not use this file except in compliance with the License.\n",              "// You may obtain a copy of the License at\n",              "//\n",              "//      http://www.apache.org/licenses/LICENSE-2.0\n",              "//\n",              "// Unless required by applicable law or agreed to in writing, software\n",              "// distributed under the License is distributed on an \"AS IS\" BASIS,\n",              "// WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.\n",              "// See the License for the specific language governing permissions and\n",              "// limitations under the License.\n",              "\n",              "/**\n",              " * @fileoverview Helpers for google.colab Python module.\n",              " */\n",              "(function(scope) {\n",              "function span(text, styleAttributes = {}) {\n",              "  const element = document.createElement('span');\n",              "  element.textContent = text;\n",              "  for (const key of Object.keys(styleAttributes)) {\n",              "    element.style[key] = styleAttributes[key];\n",              "  }\n",              "  return element;\n",              "}\n",              "\n",              "// Max number of bytes which will be uploaded at a time.\n",              "const MAX_PAYLOAD_SIZE = 100 * 1024;\n",              "\n",              "function _uploadFiles(inputId, outputId) {\n",              "  const steps = uploadFilesStep(inputId, outputId);\n",              "  const outputElement = document.getElementById(outputId);\n",              "  // Cache steps on the outputElement to make it available for the next call\n",              "  // to uploadFilesContinue from Python.\n",              "  outputElement.steps = steps;\n",              "\n",              "  return _uploadFilesContinue(outputId);\n",              "}\n",              "\n",              "// This is roughly an async generator (not supported in the browser yet),\n",              "// where there are multiple asynchronous steps and the Python side is going\n",              "// to poll for completion of each step.\n",              "// This uses a Promise to block the python side on completion of each step,\n",              "// then passes the result of the previous step as the input to the next step.\n",              "function _uploadFilesContinue(outputId) {\n",              "  const outputElement = document.getElementById(outputId);\n",              "  const steps = outputElement.steps;\n",              "\n",              "  const next = steps.next(outputElement.lastPromiseValue);\n",              "  return Promise.resolve(next.value.promise).then((value) => {\n",              "    // Cache the last promise value to make it available to the next\n",              "    // step of the generator.\n",              "    outputElement.lastPromiseValue = value;\n",              "    return next.value.response;\n",              "  });\n",              "}\n",              "\n",              "/**\n",              " * Generator function which is called between each async step of the upload\n",              " * process.\n",              " * @param {string} inputId Element ID of the input file picker element.\n",              " * @param {string} outputId Element ID of the output display.\n",              " * @return {!Iterable<!Object>} Iterable of next steps.\n",              " */\n",              "function* uploadFilesStep(inputId, outputId) {\n",              "  const inputElement = document.getElementById(inputId);\n",              "  inputElement.disabled = false;\n",              "\n",              "  const outputElement = document.getElementById(outputId);\n",              "  outputElement.innerHTML = '';\n",              "\n",              "  const pickedPromise = new Promise((resolve) => {\n",              "    inputElement.addEventListener('change', (e) => {\n",              "      resolve(e.target.files);\n",              "    });\n",              "  });\n",              "\n",              "  const cancel = document.createElement('button');\n",              "  inputElement.parentElement.appendChild(cancel);\n",              "  cancel.textContent = 'Cancel upload';\n",              "  const cancelPromise = new Promise((resolve) => {\n",              "    cancel.onclick = () => {\n",              "      resolve(null);\n",              "    };\n",              "  });\n",              "\n",              "  // Wait for the user to pick the files.\n",              "  const files = yield {\n",              "    promise: Promise.race([pickedPromise, cancelPromise]),\n",              "    response: {\n",              "      action: 'starting',\n",              "    }\n",              "  };\n",              "\n",              "  cancel.remove();\n",              "\n",              "  // Disable the input element since further picks are not allowed.\n",              "  inputElement.disabled = true;\n",              "\n",              "  if (!files) {\n",              "    return {\n",              "      response: {\n",              "        action: 'complete',\n",              "      }\n",              "    };\n",              "  }\n",              "\n",              "  for (const file of files) {\n",              "    const li = document.createElement('li');\n",              "    li.append(span(file.name, {fontWeight: 'bold'}));\n",              "    li.append(span(\n",              "        `(${file.type || 'n/a'}) - ${file.size} bytes, ` +\n",              "        `last modified: ${\n",              "            file.lastModifiedDate ? file.lastModifiedDate.toLocaleDateString() :\n",              "                                    'n/a'} - `));\n",              "    const percent = span('0% done');\n",              "    li.appendChild(percent);\n",              "\n",              "    outputElement.appendChild(li);\n",              "\n",              "    const fileDataPromise = new Promise((resolve) => {\n",              "      const reader = new FileReader();\n",              "      reader.onload = (e) => {\n",              "        resolve(e.target.result);\n",              "      };\n",              "      reader.readAsArrayBuffer(file);\n",              "    });\n",              "    // Wait for the data to be ready.\n",              "    let fileData = yield {\n",              "      promise: fileDataPromise,\n",              "      response: {\n",              "        action: 'continue',\n",              "      }\n",              "    };\n",              "\n",              "    // Use a chunked sending to avoid message size limits. See b/62115660.\n",              "    let position = 0;\n",              "    do {\n",              "      const length = Math.min(fileData.byteLength - position, MAX_PAYLOAD_SIZE);\n",              "      const chunk = new Uint8Array(fileData, position, length);\n",              "      position += length;\n",              "\n",              "      const base64 = btoa(String.fromCharCode.apply(null, chunk));\n",              "      yield {\n",              "        response: {\n",              "          action: 'append',\n",              "          file: file.name,\n",              "          data: base64,\n",              "        },\n",              "      };\n",              "\n",              "      let percentDone = fileData.byteLength === 0 ?\n",              "          100 :\n",              "          Math.round((position / fileData.byteLength) * 100);\n",              "      percent.textContent = `${percentDone}% done`;\n",              "\n",              "    } while (position < fileData.byteLength);\n",              "  }\n",              "\n",              "  // All done.\n",              "  yield {\n",              "    response: {\n",              "      action: 'complete',\n",              "    }\n",              "  };\n",              "}\n",              "\n",              "scope.google = scope.google || {};\n",              "scope.google.colab = scope.google.colab || {};\n",              "scope.google.colab._files = {\n",              "  _uploadFiles,\n",              "  _uploadFilesContinue,\n",              "};\n",              "})(self);\n",              "</script> "            ]          },          "metadata": {}        },        {          "output_type": "stream",          "name": "stdout",          "text": [            "Saving policydocument.pdf to policydocument.pdf\n"          ]        }      ]    },    {      "cell_type": "code",      "source": [        "pdf_paths = [f\"/content/{filename}\" for filename in uploaded.keys()]\n",        "print(f\"selected PDFs: {pdf_paths}\")"      ],      "metadata": {        "colab": {          "base_uri": "https://localhost:8080/"        },        "id": "2mx48i0EsXkg",        "outputId": "9e6140bd-6a91-4b6b-9bb5-9c38c81d52d5"      },      "execution_count": null,      "outputs": [        {          "output_type": "stream",          "name": "stdout",          "text": [            "selected PDFs: ['/content/policydocument.pdf']\n"          ]        }      ]    },    {      "cell_type": "code",      "source": [        "def get_pdf_text(pdf_docs):\n",        "  '''\n",        "  loop over all the pdf files and concatenate\n",        "  the content in a single string\n",        "  '''\n",        "  text = \"\"\n",        "  for pdf in pdf_paths:\n",        "    pdf_reader = PdfReader(pdf)\n",        "    for page in pdf_reader.pages:\n",        "      page_text = page.extract_text()\n",        "      if page_text:\n",        "        text += page_text\n",        "    return text\n"      ],      "metadata": {        "id": "XqaqoAxuseNX"      },      "execution_count": null,      "outputs": []    },    {      "cell_type": "code",      "source": [        "raw_text = get_pdf_text(pdf_paths)\n",        "print(\"✅ Extracted text length:\", len(raw_text))"      ],      "metadata": {        "colab": {          "base_uri": "https://localhost:8080/"        },        "id": "lT-b1hufshBM",        "outputId": "9f0df21c-b48d-456d-fd8c-0a93568a66a2"      },      "execution_count": null,      "outputs": [        {          "output_type": "stream",          "name": "stdout",          "text": [            "✅ Extracted text length: 876554\n"          ]        }      ]    },    {      "cell_type": "code",      "source": [        "def get_text_chunks(raw_text):\n",        "  '''\n",        "  takes a single string of text and returns a list\n",        "  of text strings that can be fed to vector database\n",        "  '''\n",        "  text_splitter = CharacterTextSplitter(\n",        "      separator=\"\\n\",\n",        "      chunk_size=1000,\n",        "      chunk_overlap=200,\n",        "      length_function=len\n",        "  )\n",        "  chunks = text_splitter.split_text(raw_text)\n",        "  return chunks\n"      ],      "metadata": {        "id": "1P9W5mIFsjsX"      },      "execution_count": null,      "outputs": []    },    {      "cell_type": "code",      "source": [        "text_chunks = get_text_chunks(raw_text)\n",        "print(\"✅ Number of text chunks:\", len(text_chunks))"      ],      "metadata": {        "colab": {          "base_uri": "https://localhost:8080/"        },        "id": "Q-EHMzYXsmVK",        "outputId": "986025fc-056f-4362-e455-1458c2d5bfc7"      },      "execution_count": null,      "outputs": [        {          "output_type": "stream",          "name": "stdout",          "text": [            "✅ Number of text chunks: 1093\n"          ]        }      ]    },    {      "cell_type": "code",      "source": [        "def get_vectorstore(text_chunks):\n",        "  '''\n",        "  creat a FAISS vectorstore from the text chunks using embeddings\n",        "  '''\n",        "  embeddings = HuggingFaceEmbeddings(\n",        "      model_name=\"hkunlp/instructor-xl\",\n",        "      model_kwargs={\"device\": \"cuda\"}\n",        "  )\n",        "  vectorstore = FAISS.from_texts(\n",        "      texts=text_chunks,\n",        "      embedding=embeddings\n",        "  )\n",        "  return vectorstore\n"      ],      "metadata": {        "id": "a69OPJ9osrC0"      },      "execution_count": null,      "outputs": []    },    {      "cell_type": "code",      "source": [        "vectorstore = get_vectorstore(text_chunks)\n",        "print(\"✅ vectorstore created\")"      ],      "metadata": {        "colab": {          "base_uri": "https://localhost:8080/",          "height": 606,          "referenced_widgets": [            "6187d9bc3315463599b6c057cdbb6677",            "d7c6b661e272463cbc56b53f87f33abc",            "882cd1317ae646ab85f7ed7807d84414",            "7eef6b642dc84be5bed85901c52b8b9a",            "bc72f7048f9544b6b2531714f1ff83f0",            "a9a1dbbe6c164a0cb7fca4285085fefc",            "720f0bf7849a46c8b62ec266894aee9c",            "db038b06c48f4c5cb66aec0f7d151e5d",            "eaa06dbf3355419eb3fbd8830d380b6c",            "973b635b6bb64b16b51efec17e43d206",            "6a1e95ad9bae4feaaddf65b2456e4423",            "30e40062550f4fa0ae0f384d2925ece6",            "7610f4069f4e4daeaff9b986e67b0d1f",            "18314b1870f24774afb973fdbd03b491",            "bff75da6cd2a41c9b42bd0e398070311",            "ba0259a97a2248eaa48d42453156bfc8",            "4b4ec49da1224c79bd9b48dadf32aba1",            "79087128df004e36a6fa6555c701fed7",            "2cf5615646ab44be81bb5d24dcd71ffa",            "9e5363a9269a4db19e89217c69f71bc9",            "5546306cb5d44e4aa1990e16cc528cfb",            "b581fb6120f44472b280ee21245b8a2f",            "ed5f0f474a724d9787bb869c8858660b",            "a81606e7465b4b539f1adb711b60a0b5",            "d16e34e14f72466fb71aec1afb7c6ae9",            "1e76695042ff447a9ee524a332edf1d6",            "a658ff4f08934270838e0a0d8d81ebf4",            "687bd28291504e7db61ec00f381b056a",            "39c24019a24b47ca9cc22a549ed2c92d",            "420af6cbf08e4143945b60c5177b38b4",            "ccc0e88464bf43e8bde63966e7ae9c57",            "7f3739e5807c4a1484801a6d231b4b1c",            "d4fb0b35c6954c7dab1ed5becb8c8141",            "acd1bcda3bca4c30a9787f47f6267383",            "d3830d8299cb427f98d210867c40b03e",            "b17e87e86068463b81823634d795ad4d",            "c23381aba2b54442840843488d2b3b34",            "b8e1668a7dbc41939b6cdd1b7ed55050",            "3870cc0850184ac98dfe1fe646d19acc",            "18d59808f7ab4ff7a5a69fcfe245cb91",            "3faa24a0c6ad4138b19447a9cae1078a",            "a4e67a6f695448e884eb62395f1ab389",            "da00e3d12c1d420091d8ba701184e51c",            "db2c37fc93ba4118ba24517a5269d3b0",            "cd3080bbee28450b98dc45d638ddcee2",            "471f0d5c20474d0c92cd376dafc27a8a",            "32980f43684f4322ae7f7b10be808d94",            "98c67eb20001423e8e8ff6ee073409da",            "f5245838df4d4471b6c9b9659bce8147",            "a131a1ad4dd84fbf9a11e48500c5e7a8",            "c236f4bc518444b6b458cb6995eb04fa",            "1d32301d8d3c438b989077f1f164f54b",            "8df0689088cb40518c97187d22585054",            "82f91f0fd4fe4af4928b016ba2c6344d",            "ff0d41c19f0440debf3aaad60c433877",            "3f5b915dff9c4c3f9676d4009da962ad",            "112c143b55a34f9b9aed31679dd66d6b",            "fcf1c33e86654e2386f6960ee71adf4a",            "298d6134df294737aaf49bb1fd0eff6b",            "ac987e8442ef41488f706b220ab89a67",            "19faa389154a4fdf8d04c1458401655f",            "4a1c6d3c532b427ebabd3d6fce842ad3",            "2ff05460fad440a3827b0d7f84f1ff81",            "1f99d6b0fb1d4ec8b43d9ab9063875b1",            "6fe5f5ec79e34c27895fe5cac6489cb7",            "3307bde944a541aaae3bbcb14a369675",            "58bbeaf2cfca4a79b01aa6f7ac5e8932",            "96eb85549d8641a28ea82e680b9f8964",            "61b7d2c5d76f4dd48d1bf091998082a1",            "a0d067955126420db4af9faf90184979",            "439df49e298b43618008f7198892aca6",            "438b614ece2c4f8cb177dd9ed487a3fe",            "e672670fd7904add8e20f35aa1aa7cd0",            "0d737cffa94d464db9e9d17b434bd2df",            "44c2f266274647edb4c0a1dcc744d2ae",            "78a910d28070429f8769a664688905be",            "44bd5016067d42a19f6137200c3fa8ac",            "07eae94ddf2745129a3c63d0d3d66f27",            "7b9df8eb955f4bb7b4c4417bc959faa6",            "99a179cdf26c4c3c8f08e8af732e7f5c",            "fbc432f9254b48e4a01fcd3d618b2a17",            "ea385f44d0094416b5a26727da8fd7bc",            "c0ff09d6a4164de8bcc66544527ff79c",            "d6ecc886777d499e8652ec1d042229a4",            "9f751a279f8a42e4bcbdd3a8009118da",            "7a6e795cd6c9425d9f0cebbbcef65b24",            "73528a41ad52491abccdbd3b93f840ba",            "95c884653bf94de7a7adba915f1a76ab",            "10f9babd755e4ca0bb36175c34321bee",            "80286c74df584858af8862bc283b32f3",            "ad94fa23988243eea29d31e3ab816773",            "567e300b6ab74ab0b8d793e8c6559483",            "e5a556ff241c4621982b2b8d57e295a7",            "e2ca1a1eea0442819f120b66dcd83053",            "cc7c897f01cd45338197b4680898a70c",            "4c9e9052f056489d9ae3917ea04a1634",            "549cff618a2a4a0182fe9f4401619207",            "064b297dac5c4067a987bdfdc03988bc",            "b660155270d644e4abb186fda004466a",            "ed16a3bdc8d84e49829d20b75e80f563",            "b9fbce71445746abbb697c9e3850a0fc",            "17ddaee8d2374e61a745ad51f43d71fc",            "40543ac91c4a4a4da8b0bda217c4ccb7",            "9759b9c3826b40a5aa5098578edfa618",            "659ed177f79b49e2bc711021e30f055a",            "38e888ac88a04f1d8f6a99a5732824b9",            "b99eeb16bc5f4c3cb223ba3d54b8d2db",            "4fe9dd10249c4e5a9349d38b48061e14",            "636104fca6674fb2942af2acc9e74b97",            "3a3b8a7a65e649cea8c7103b20f425bf",            "7d4a64d4c6ba4ff8ab4e3679be3d5a37",            "94651b4dd4f546789824746c975620f2",            "cc526b0d5e314c9a9bb107280fe3bad8",            "85e3422561b242fca101fff36f3ab433",            "8dc69bdd76e946a3814209baeae194e5",            "75767740f74240fabd0dd78b666bdd2f",            "a9d68dab46824807a060eb5037acf196",            "ef0543f80e144a41bc5c75b7d07de8b4",            "6c594f7ae16a4a409579bb8d6d032eeb",            "4f5d19e579f14534b64f3b2daa7b65f9",            "9d82748ecfcd4d99adddc944f45fd3d5",            "2a203a71ab9543f08187efc4727ccee2",            "e7622d65cf694568bc9aac46e3b0efca",            "3e85c144268e43a08f15c5b3310765cf",            "5247110dfd7943cd96f9d333747f71bd",            "c145cf17ce8b4f3cad7da30fdb68eadc",            "a1bbd4bac87c439992c4b3e156d48a78",            "ad5203da66eb4509a8bf5e1e1c18dad5",            "0a54c2ce5dca4b7f9976dbb0278d63b8",            "5a88f3855be441fbbba300b3a48fa9bd",            "ae396983970b4950a018fa50e66ee5e7",            "ed0a0ec9ea824945bf3dda0b00caee2a",            "32c2a009a0d444fcb489b63fdd31833d",            "96266112a96c4d318f20cad78214aca8",            "2dcd5e47be3940109eedb0ba23cef2e1",            "11d80024a77943d08d149f5812266a7d",            "2cc0bb4de49844f8a104d0845fa76b54",            "dde67741029c487fb3dc5a5c7e099a3c",            "125b611f34bb4302be41c287983aee7c",            "fbe250ffa27c445ea253f64cc0afbd83",            "5a91238b1ef341c487c546469953322d",            "b4774dbe487342aa9ddbe41fd10fd685",            "474f6e0a06e341e197fc79f4008d0572",            "508f236bf682419cbafbb5ce63e0f515",            "d9cca8d159274e4cae1f94400813dff6",            "3624b56fb27248288826896c61934d58",            "4c38048133024510be7cd143252c6ceb",            "0e51d089c63e4e1aaf2e874c271a2b98",            "8af033d4969e4f348ed8532e5a0e2817",            "c25b81d0d41245b28b7ada2219079111",            "11dc04dab55044058f6a4eecc7c70d82",            "030e9279a594403aaf19b48f5470e116",            "5bf8cedb2a90426895d601e7a3bd1bb1",            "bd08a37cff63452dbe6c58965cbbbcd2"          ]        },        "id": "N73JLRYsssNs",        "outputId": "fa4dcfb6-4327-4871-92aa-341101e0877a"      },      "execution_count": null,      "outputs": [        {          "output_type": "stream",          "name": "stderr",          "text": [            "/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: \n",            "The secret `HF_TOKEN` does not exist in your Colab secrets.\n",            "To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.\n",            "You will be able to reuse this secret in all of your notebooks.\n",            "Please note that authentication is recommended but still optional to access public models or datasets.\n",            "  warnings.warn(\n"          ]        },        {          "output_type": "display_data",          "data": {            "text/plain": [              "modules.json:   0%|          | 0.00/461 [00:00<?, ?B/s]"            ],            "application/vnd.jupyter.widget-view+json": {              "version_major": 2,              "version_minor": 0,              "model_id": "6187d9bc3315463599b6c057cdbb6677"            }          },          "metadata": {}        },        {          "output_type": "display_data",          "data": {            "text/plain": [              "config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]"            ],            "application/vnd.jupyter.widget-view+json": {              "version_major": 2,              "version_minor": 0,              "model_id": "30e40062550f4fa0ae0f384d2925ece6"            }          },          "metadata": {}        },        {          "output_type": "display_data",          "data": {            "text/plain": [              "README.md: 0.00B [00:00, ?B/s]"            ],            "application/vnd.jupyter.widget-view+json": {              "version_major": 2,              "version_minor": 0,              "model_id": "ed5f0f474a724d9787bb869c8858660b"            }          },          "metadata": {}        },        {          "output_type": "display_data",          "data": {            "text/plain": [              "sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]"            ],            "application/vnd.jupyter.widget-view+json": {              "version_major": 2,              "version_minor": 0,              "model_id": "acd1bcda3bca4c30a9787f47f6267383"            }          },          "metadata": {}        },        {          "output_type": "display_data",          "data": {            "text/plain": [              "config.json: 0.00B [00:00, ?B/s]"            ],            "application/vnd.jupyter.widget-view+json": {              "version_major": 2,              "version_minor": 0,              "model_id": "cd3080bbee28450b98dc45d638ddcee2"            }          },          "metadata": {}        },        {          "output_type": "display_data",          "data": {            "text/plain": [              "pytorch_model.bin:   0%|          | 0.00/4.96G [00:00<?, ?B/s]"            ],            "application/vnd.jupyter.widget-view+json": {              "version_major": 2,              "version_minor": 0,              "model_id": "3f5b915dff9c4c3f9676d4009da962ad"            }          },          "metadata": {}        },        {          "output_type": "display_data",          "data": {            "text/plain": [              "model.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]"            ],            "application/vnd.jupyter.widget-view+json": {              "version_major": 2,              "version_minor": 0,              "model_id": "58bbeaf2cfca4a79b01aa6f7ac5e8932"            }          },          "metadata": {}        },        {          "output_type": "display_data",          "data": {            "text/plain": [              "tokenizer_config.json: 0.00B [00:00, ?B/s]"            ],            "application/vnd.jupyter.widget-view+json": {              "version_major": 2,              "version_minor": 0,              "model_id": "07eae94ddf2745129a3c63d0d3d66f27"            }          },          "metadata": {}        },        {          "output_type": "display_data",          "data": {            "text/plain": [              "spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]"            ],            "application/vnd.jupyter.widget-view+json": {              "version_major": 2,              "version_minor": 0,              "model_id": "10f9babd755e4ca0bb36175c34321bee"            }          },          "metadata": {}        },        {          "output_type": "display_data",          "data": {            "text/plain": [              "tokenizer.json: 0.00B [00:00, ?B/s]"            ],            "application/vnd.jupyter.widget-view+json": {              "version_major": 2,              "version_minor": 0,              "model_id": "ed16a3bdc8d84e49829d20b75e80f563"            }          },          "metadata": {}        },        {          "output_type": "display_data",          "data": {            "text/plain": [              "special_tokens_map.json: 0.00B [00:00, ?B/s]"            ],            "application/vnd.jupyter.widget-view+json": {              "version_major": 2,              "version_minor": 0,              "model_id": "7d4a64d4c6ba4ff8ab4e3679be3d5a37"            }          },          "metadata": {}        },        {          "output_type": "display_data",          "data": {            "text/plain": [              "config.json:   0%|          | 0.00/270 [00:00<?, ?B/s]"            ],            "application/vnd.jupyter.widget-view+json": {              "version_major": 2,              "version_minor": 0,              "model_id": "2a203a71ab9543f08187efc4727ccee2"            }          },          "metadata": {}        },        {          "output_type": "display_data",          "data": {            "text/plain": [              "pytorch_model.bin:   0%|          | 0.00/3.15M [00:00<?, ?B/s]"            ],            "application/vnd.jupyter.widget-view+json": {              "version_major": 2,              "version_minor": 0,              "model_id": "32c2a009a0d444fcb489b63fdd31833d"            }          },          "metadata": {}        },        {          "output_type": "display_data",          "data": {            "text/plain": [              "config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]"            ],            "application/vnd.jupyter.widget-view+json": {              "version_major": 2,              "version_minor": 0,              "model_id": "508f236bf682419cbafbb5ce63e0f515"            }          },          "metadata": {}        },        {          "output_type": "stream",          "name": "stdout",          "text": [            "✅ vectorstore created\n"          ]        }      ]    },    {      "cell_type": "code",      "source": [        "def get_conversation_chain(vectorstore):\n",        "    '''\n",        "    Creates a conversational retrieval chain using OpenAI's mode\n",        "    '''\n",        "   #llm = ChatOpenAI(\n",        "     # model_name= \"gpt-4o-mini\",\n",        "     # temperature=0.7,\n",        "     # request_timeout=30\n",        "    llm = ChatGroq(\n",        "         model_name=\"llama-3.3-70b-versatile\",\n",        "         temperature=0.7,\n",        "         request_timeout=30,\n",        "         api_key=\"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\"\n",        "    )\n",        "    memory = ConversationBufferMemory(\n",        "        memory_key='chat_history',\n",        "        return_messages=True\n",        "    )\n",        "    conversation_chain = ConversationalRetrievalChain.from_llm(\n",        "        llm=llm,\n",        "        retriever=vectorstore.as_retriever(),\n",        "        memory=memory\n",        "    )\n",        "    return conversation_chain\n"      ],      "metadata": {        "id": "bzFvH464svEU"      },      "execution_count": null,      "outputs": []    },    {      "cell_type": "code",      "source": [        "conversation = get_conversation_chain(vectorstore)"      ],      "metadata": {        "colab": {          "base_uri": "https://localhost:8080/"        },        "id": "_IrZblT0s2hV",        "outputId": "929627a7-cdc7-4498-c623-e6722b78050d"      },      "execution_count": null,      "outputs": [        {          "output_type": "stream",          "name": "stderr",          "text": [            "/tmp/ipython-input-16-2086005007.py:15: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/\n",            "  memory = ConversationBufferMemory(\n"          ]        }      ]    },    {      "cell_type": "code",      "source": [        "def chat_with_pdf(question):\n",        "    response = conversation({'question': question})\n",        "    for i, msg in enumerate(response['chat_history']):\n",        "      sender = \"User\" if i % 2 == 0 else \"BOT\"\n",        "      print(f\"{sender}: {msg.content}\")"      ],      "metadata": {        "id": "Gtqfjt01s3Q4"      },      "execution_count": null,      "outputs": []    },    {      "cell_type": "code",      "source": [        "chat_with_pdf(\"What is constitution?\")"      ],      "metadata": {        "colab": {          "base_uri": "https://localhost:8080/"        },        "id": "UKHu_UU7s51B",        "outputId": "1fb39bd3-44d0-4e06-db16-d2db7bbce034"      },      "execution_count": null,      "outputs": [        {          "output_type": "stream",          "name": "stderr",          "text": [            "/tmp/ipython-input-18-206198848.py:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.\n",            "  response = conversation({'question': question})\n"          ]        },        {          "output_type": "stream",          "name": "stdout",          "text": [            "User: What is constitution?\n",            "BOT: The Constitution refers to the supreme law or the foundational document of a country that outlines the framework, principles, and powers of its government, as well as the rights and responsibilities of its citizens. In the context of the provided text, the Constitution of India is the supreme law of India that establishes the country as a sovereign, socialist, secular, democratic republic and outlines the principles of justice, liberty, equality, and fraternity for its citizens.\n"          ]        }      ]    },    {      "cell_type": "code",      "source": [        "chat_with_pdf(\"what are the fundamental rights?\")"      ],      "metadata": {        "colab": {          "base_uri": "https://localhost:8080/"        },        "id": "CbZsLZKes8yJ",        "outputId": "7192d5bc-2327-489c-9c68-8d51e793c370"      },      "execution_count": null,      "outputs": [        {          "output_type": "stream",          "name": "stdout",          "text": [            "User: What is constitution?\n",            "BOT: The Constitution refers to the supreme law or the foundational document of a country that outlines the framework, principles, and powers of its government, as well as the rights and responsibilities of its citizens. In the context of the provided text, the Constitution of India is the supreme law of India that establishes the country as a sovereign, socialist, secular, democratic republic and outlines the principles of justice, liberty, equality, and fraternity for its citizens.\n",            "User: what are the fundamental rights?\n",            "BOT: The fundamental rights are listed under Part III of the Constitution of India. They include:\n",            "\n",            "1. Right to Equality (Articles 14-18):\n",            "   - Equality before law (Article 14)\n",            "   - Prohibition of discrimination on grounds of religion, race, caste, sex or place of birth (Article 15)\n",            "   - Equality of opportunity in matters of public employment (Article 16)\n",            "   - Abolition of Untouchability (Article 17)\n",            "   - Abolition of titles (Article 18)\n",            "\n",            "2. Right to Freedom (Articles 19-22):\n",            "   - Protection of certain rights regarding freedom of speech, etc. (Article 19)\n",            "   - Protection in respect of conviction for offences (Article 20)\n",            "   - Protection of life and personal liberty (Article 21)\n",            "   - Right to education (Article 21A)\n",            "   - Protection against arrest and detention in certain cases (Article 22)\n",            "\n",            "These rights are guaranteed to all citizens of India and are enforceable by the courts, with some limitations and exceptions.\n"          ]        }      ]    },    {      "cell_type": "code",      "source": [        "chat_with_pdf(\"what are the fundamental rights of women safety?\")"      ],      "metadata": {        "colab": {          "base_uri": "https://localhost:8080/"        },        "id": "rEjDZTXbyQFI",        "outputId": "1f2de6e4-da8e-4632-9064-6bbdc8640341"      },      "execution_count": null,      "outputs": [        {          "output_type": "stream",          "name": "stdout",          "text": [            "User: What is constitution?\n",            "BOT: The Constitution refers to the supreme law or the foundational document of a country that outlines the framework, principles, and powers of its government, as well as the rights and responsibilities of its citizens. In the context of the provided text, the Constitution of India is the supreme law of India that establishes the country as a sovereign, socialist, secular, democratic republic and outlines the principles of justice, liberty, equality, and fraternity for its citizens.\n",            "User: what are the fundamental rights?\n",            "BOT: The fundamental rights are listed under Part III of the Constitution of India. They include:\n",            "\n",            "1. Right to Equality (Articles 14-18):\n",            "   - Equality before law (Article 14)\n",            "   - Prohibition of discrimination on grounds of religion, race, caste, sex or place of birth (Article 15)\n",            "   - Equality of opportunity in matters of public employment (Article 16)\n",            "   - Abolition of Untouchability (Article 17)\n",            "   - Abolition of titles (Article 18)\n",            "\n",            "2. Right to Freedom (Articles 19-22):\n",            "   - Protection of certain rights regarding freedom of speech, etc. (Article 19)\n",            "   - Protection in respect of conviction for offences (Article 20)\n",            "   - Protection of life and personal liberty (Article 21)\n",            "   - Right to education (Article 21A)\n",            "   - Protection against arrest and detention in certain cases (Article 22)\n",            "\n",            "These rights are guaranteed to all citizens of India and are enforceable by the courts, with some limitations and exceptions.\n",            "User: what are the fundamental rights of women safety?\n",            "BOT: The Constitution of India has several provisions that ensure women's safety and equality. Some of the key provisions include:\n",            "\n",            "1. **Article 14**: Equality before the law - This article ensures that all citizens, including women, are equal before the law and have equal protection of the law.\n",            "2. **Article 15**: Prohibition of discrimination on grounds of sex - This article prohibits discrimination against women on the basis of sex.\n",            "3. **Article 15(3)**: Special provisions for women - This article allows the State to make special provisions for women, including provisions for their safety and welfare.\n",            "4. **Article 16**: Equality of opportunity in matters of public employment - This article ensures that women have equal opportunities in public employment.\n",            "5. **Article 21**: Protection of life and personal liberty - This article ensures that women have the right to life and personal liberty, and protects them from violence and exploitation.\n",            "6. **Article 39**: Certain principles of policy to be followed by the State - This article directs the State to ensure that women are not forced into occupations that are not suitable for them, and that they are protected from exploitation.\n",            "7. **Article 39A**: Equal justice and free legal aid - This article ensures that women have access to equal justice and free legal aid.\n",            "8. **Article 42**: Provision for just and humane conditions of work - This article ensures that women are protected from exploitation and have just and humane conditions of work.\n",            "\n",            "Additionally, the Constitution has been amended to include provisions such as:\n",            "\n",            "1. **The 73rd and 74th Constitutional Amendments**: These amendments reserved one-third of the seats in local self-government institutions, such as municipalities and panchayats, for women.\n",            "2. **The Protection of Women from Domestic Violence Act, 2005**: This act provides protection to women from domestic violence and provides for emergency relief and support services.\n",            "3. **The Sexual Harassment of Women at Workplace (Prevention, Prohibition and Redressal) Act, 2013**: This act provides protection to women from sexual harassment at the workplace and provides for a complaint mechanism and penalties for offenders.\n",            "\n",            "These constitutional provisions and laws aim to ensure women's safety and equality in India, but their implementation and effectiveness vary across different regions and contexts.\n"          ]        }      ]    }  ]}'''

with open('rag_fastapi_app.py', 'w') as f:
    f.write(app_code)

print('✅ rag_fastapi_app.py created successfully')

In [ ]:
# Run the FastAPI app in the background
uvicorn.run('rag_fastapi_app:app', host='0.0.0.0', port=8000)